# Category1. 미세먼지 저감효과

### 목차
- (1) 노출 빈도
  
- (2) 노출 심도
    - 1 도로재비산
    - 2 공장 
    - 3 교통량, 도로재비산 정류장 연결
- (3) 최종 결합
    - 미세먼지 순위

- (4) 시각화

- (5) 군집화
    -  K-means 모델링
        - 최적의 K찾기
        - seaborn 시각화
        - 계층적 클러스터링

<사용된 라이브러리>

In [1]:
import geopandas as gpd
import numpy as np
import pandas as pd
import json

import re 
import datetime as dt 

import scipy as sp
import seaborn as sns
import pyproj

from fiona.crs import from_epsg
import requests
import branca 
from branca.colormap import linear
import folium
folium.__version__
from folium import plugins

from shapely.geometry import Polygon, LineString, Point
from tqdm import tqdm
import shapely


import urllib
import urllib.request

/opt/app-root/lib/python3.6/site-packages/geopandas/_compat.py:91: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.0-CAPI-1.16.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [2]:
from geoband.API import *
GetCompasData('SBJ_2102_002', '1', '1.수원시_버스정류장.csv')
GetCompasData('SBJ_2102_002', '2', '2.수원시_버스정류장별_승하차이력(1).csv')
GetCompasData('SBJ_2102_002', '3', '3.수원시_버스정류장별_승하차이력(2).csv')
GetCompasData('SBJ_2102_002', '4', '4.수원시_버스정류장별_승하차이력(3).csv')
GetCompasData('SBJ_2102_002', '5', '5.수원시_버스정류장별_승하차이력(4).csv')
GetCompasData('SBJ_2102_002', '6', '6.수원시_버스정류장별_승하차이력(5).csv')
GetCompasData('SBJ_2102_002', '22', '22.수원시_상세도로망_LV6.geojson')
GetCompasData('SBJ_2102_002', '23', '23.수원시_평일_일별_시간대별_추정교통량_LV6.csv')
GetCompasData('SBJ_2102_002', '29', '29.수원시_법정경계(시군구).geojson')
GetCompasData('SBJ_2102_002', '30', '30.수원시_법정경계(읍면동).geojson')

### (1) 노출빈도
- 정류장별 노선별 승차인원 * 대기시간(배차간격)

In [3]:
busstop1=gpd.read_file('2.수원시_버스정류장별_승하차이력(1).csv')
busstop2=gpd.read_file('3.수원시_버스정류장별_승하차이력(2).csv')
busstop3=gpd.read_file('4.수원시_버스정류장별_승하차이력(3).csv')
busstop4=gpd.read_file('5.수원시_버스정류장별_승하차이력(4).csv')
busstop5=gpd.read_file('6.수원시_버스정류장별_승하차이력(5).csv')

- 노선정보 ( 노선별 배차간격)
-  http://www.gbis.go.kr/service/busInfo/metroBus.action?cmd=metroBusList 에서 경유 노선, 마을버스, 지역버스 인허가 노선 사용

In [4]:
buswait1=pd.read_excel('data/경유노선.xlsx')
buswait2=pd.read_excel('data/마을버스인허가노선.xlsx')
buswait3=pd.read_excel('data/지역버스인허가노선.xlsx')

In [5]:
buswait1.columns=buswait1.iloc[4,]
buswait1=buswait1.iloc[5:,]
buswait1.index=buswait1['순번']
buswait1.drop(['순번'],axis=1,inplace=True)

In [6]:
buswait2.columns=buswait2.iloc[4,]
buswait2=buswait2.iloc[5:,]
buswait2.index=buswait2['순번']
buswait2.drop(['순번'],axis=1,inplace=True)
buswait2.head(5)

4  관할관청       운행업체 노선번호          기점                    종점  인가거리 인가대수 출퇴근배차  \
순번                                                                           
1   수원시    수원-광교운수    1       우만3단지         광교중앙.아주대역환승센터    16    0    20   
2   수원시    수원-광교운수    2  호반마을광교21단지                 경기대후문  28.6    0    19   
3   수원시  수원-율전마을버스  2-1     동원고(마을)               조원주공아파트  16.2    0     5   
4   수원시  수원-율전마을버스  2-5     동원고(마을)  STX칸아파트.현대힐스테이트204동앞  13.9    0     8   
5   수원시    수원-동방운수   21      계명고등학교              권선꿈에그린동문  28.7    0    10   

4  평일배차 주말배차 주중상행첫차 주중상행막차 주말상행첫차 주말상행막차 주중하행첫차 주중하행막차 주말하행첫차 주말하행막차   비고  
순번                                                                         
1    45   50   6:00  22:30   6:00  22:30   6:00  22:30   6:00  22:30  NaN  
2    45   55   6:00  22:30   6:00  22:22   6:00  22:30   6:00  22:22  NaN  
3    14   15   6:00  23:30   6:00  23:30   6:00  23:40   6:00  23:30  NaN  
4    15   25   6:00  22:30   6:00  23:30   8:40  23:30   8:00  23:30  NaN  
5    15   20   6:00  22:30   6:00  22:30   6:00  22:30   6:30  22:30  NaN

In [7]:
buswait3.columns=buswait3.iloc[4,]
buswait3=buswait3.iloc[5:,]
buswait3.index=buswait3['순번']
buswait3.drop(['순번'],axis=1,inplace=True)
buswait3.head(5)

4  관할관청  운행업체    노선번호       기점                  종점  인가거리 인가대수 출퇴근배차 평일배차 주말배차  \
순번                                                                              
1   수원시  대원고속  1007-1    대광빌리지            잠실역.롯데월드  74.6    5    40   50   60   
2   수원시  수원여객    11-1  곡반정동차고지      수원역환승센터(10승강장)  64.9   15    15   20   26   
3   수원시  대원고속    1112   경희대차고지       테크노마트앞.강변역(C)  86.6   16    10   20   30   
4   수원시  수원여객     112  곡반정동차고지               웅비아파트  38.7   22     7    9   12   
5   수원시  용남고속    123A   경기도청입구  LG빌리지2차.휴먼시아5단지아파트  31.5    3    50   60  100   

4  주중상행첫차 주중상행막차 주말상행첫차 주말상행막차 주중하행첫차 주중하행막차 주말하행첫차 주말하행막차   비고  
순번                                                               
1   05:00  22:10  05:00  22:10  06:20  23:30  06:20  23:30  NaN  
2   05:00  22:40  05:00  22:40  05:10  22:40  05:10  22:40  NaN  
3   04:40  22:30  04:40  22:30  06:00  23:50  06:00  23:50  NaN  
4   05:00  22:50  05:00  22:50  06:05  23:55  06:05  23:55  NaN  
5   06:30  22:00  06:30  22:00  07:30  23:00  07:30  23:00  NaN

- 사용할 열 선택

In [8]:
buswait1=buswait1.loc[:,['노선번호','평일배차','주말배차','출퇴근배차']]
buswait2=buswait2.loc[:,['노선번호','평일배차','주말배차','출퇴근배차']]
buswait3=buswait3.loc[:,['노선번호','평일배차','주말배차','출퇴근배차']]
buswait=pd.concat([buswait1,buswait2,buswait3],ignore_index=True)

- 평일, 주말 구분

In [9]:
weeks1=['평일' if i.weekday()<6 else '주말' for i in pd.to_datetime(busstop1['일자'])]
weeks2=['평일' if i.weekday()<6 else '주말' for i in pd.to_datetime(busstop2['일자'])]
weeks3=['평일' if i.weekday()<6 else '주말' for i in pd.to_datetime(busstop3['일자'])]
weeks4=['평일' if i.weekday()<6 else '주말' for i in pd.to_datetime(busstop4['일자'])]
weeks5=['평일' if i.weekday()<6 else '주말' for i in pd.to_datetime(busstop5['일자'])]

- 정류소별 승하차 인원 평일, 주말로 요약

In [10]:
busstop1['요일']=weeks1
busstop2['요일']=weeks2
busstop3['요일']=weeks3
busstop4['요일']=weeks4
busstop5['요일']=weeks5

busstop1.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 364448 entries, 0 to 364447
Data columns (total 14 columns):
 #   Column    Non-Null Count   Dtype   
---  ------    --------------   -----   
 0   일자        364448 non-null  object  
 1   업체명       364448 non-null  object  
 2   노선번호      364448 non-null  object  
 3   노선유형      364448 non-null  object  
 4   정류소명      364448 non-null  object  
 5   정류소ID     364448 non-null  object  
 6   전체 승차 건수  364448 non-null  object  
 7   초승 건수     364448 non-null  object  
 8   환승 건수     364448 non-null  object  
 9   전체 하차 건수  364448 non-null  object  
 10  하차 건수     364448 non-null  object  
 11  미태그 건수    364448 non-null  object  
 12  geometry  0 non-null       geometry
 13  요일        364448 non-null  object  
dtypes: geometry(1), object(13)
memory usage: 38.9+ MB


- 전체 승차 건수 int형으로 변경

In [11]:
busstop1['전체 승차 건수']=busstop1['전체 승차 건수'].astype(int)
busstop2['전체 승차 건수']=busstop2['전체 승차 건수'].astype(int)
busstop3['전체 승차 건수']=busstop3['전체 승차 건수'].astype(int)
busstop4['전체 승차 건수']=busstop4['전체 승차 건수'].astype(int)
busstop5['전체 승차 건수']=busstop5['전체 승차 건수'].astype(int)

In [12]:
bus1=busstop1.loc[:,['전체 승차 건수','정류소ID','노선번호','요일']].groupby(by=['정류소ID','노선번호','요일'],as_index=False).sum()
bus2=busstop2.loc[:,['전체 승차 건수','정류소ID','노선번호','요일']].groupby(by=['정류소ID','노선번호','요일'],as_index=False).sum()
bus3=busstop3.loc[:,['전체 승차 건수','정류소ID','노선번호','요일']].groupby(by=['정류소ID','노선번호','요일'],as_index=False).sum()
bus4=busstop4.loc[:,['전체 승차 건수','정류소ID','노선번호','요일']].groupby(by=['정류소ID','노선번호','요일'],as_index=False).sum()
bus5=busstop5.loc[:,['전체 승차 건수','정류소ID','노선번호','요일']].groupby(by=['정류소ID','노선번호','요일'],as_index=False).sum()

### 승차인원과 대기시간 데이터 결합

- 노선번호 데이터 유형이 달라서 정규표현식을 이용한 통일작업

In [13]:
bus1['노선번호']=[re.sub('-|_',' ',i) for i in bus1['노선번호']]
bus2['노선번호']=[re.sub('-|_',' ',i) for i in bus2['노선번호']]
bus3['노선번호']=[re.sub('-|_',' ',i) for i in bus3['노선번호']]
bus4['노선번호']=[re.sub('-|_',' ',i) for i in bus4['노선번호']]
bus5['노선번호']=[re.sub('-|_',' ',i) for i in bus5['노선번호']]

buswait['노선번호']=[re.sub('-|_',' ',i) for i in buswait['노선번호']]

- 불일치 항목 탐색
- 18번 버스 2021년 폐지됨.
- 18번 버스 이용객이 다른 노선으로 여전히 이용할 수 있기 때문에
- 정확한 인구측정을 위해서 제외X
- 18번 버스 정보 따로 검색해서 행 추가

In [14]:
bus=pd.concat([bus1,bus2,bus3,bus4,bus5],ignore_index=True)
iter=bus['노선번호'].unique()

for i in range(len(iter)):
     if iter[i] not in buswait['노선번호'].unique():
            print(iter[i])

18


In [15]:
buswait.loc[len(buswait)]=['18','35','38','0'] # 18번 버스 배차간격들 # 18번 버스 배차간격들
buswait.tail(5)

4         노선번호 평일배차 주말배차 출퇴근배차
285       99 2   25   30    15
286      G5100   68   68    24
287  P9100(출근)    0  NaN     0
288  P9100(퇴근)    0  NaN     0
289         18   35   38     0

- 배차간격 0 혹은 결측 데이터 전처리
    - 중복제거, 결측치 특성파악(통근형 버스, 예약가능)==> 대기시간 0분
    - 통근형 버스, 예약가능=> 대기시간0으로 대체

In [16]:
buswait=buswait.drop_duplicates('노선번호',keep='first').reset_index()
buswait=buswait.iloc[:,1:]

buswait.iloc[:,1:].loc[buswait['평일배차']=='0',:]=0
buswait['주말배차'][buswait['주말배차'].isna()]=0
buswait.iloc[:,1:]=buswait.iloc[:,1:].astype(int)

/opt/app-root/lib/python3.6/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/opt/app-root/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


### 결합(평일 승차건수x배차간격+ 주말 승차건수x배차간격)

In [17]:
bus=pd.merge(bus,buswait,how='left',left_on='노선번호',right_on='노선번호')

In [18]:
bus_weekend=bus.loc[bus['요일']=='주말',:]
bus_weekday=bus.loc[bus['요일']=='평일',:]

bus_weekday.head()

정류소ID  노선번호  요일  전체 승차 건수 평일배차 주말배차 출퇴근배차
1  200000037  3000  평일     16748   15   15    12
3  200000037  7770  평일     60374    8    7     5
4  200000037  7800  평일         0   10   10     8
6  200000041  7780  평일     19293   15   13    12
8  200000042  7780  평일     37436   15   13    12

In [19]:
bus_weekend['total']=bus_weekend['전체 승차 건수']*bus_weekend['주말배차']
bus_weekday['total']=bus_weekday['전체 승차 건수']*bus_weekday['평일배차']

a=bus_weekend.groupby(by=['정류소ID'])['total'].sum()
b=bus_weekday.groupby(by=['정류소ID'])['total'].sum()

DustIndex=a+b

print(DustIndex.shape)
print(DustIndex.describe())

/opt/app-root/lib/python3.6/site-packages/geopandas/geodataframe.py:1190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


(1114,)
count    1.114000e+03
mean     1.807189e+06
std      3.795864e+06
min      5.000000e+01
25%      1.400850e+05
50%      6.181980e+05
75%      1.927996e+06
max      4.713429e+07
Name: total, dtype: float64


- 데이터 저장

In [20]:
pd.DataFrame(DustIndex).to_csv('data/dustindex.csv',index=True)
pd.read_csv('data/dustindex.csv')

정류소ID    total
0     200000001     3731
1     200000002     6846
2     200000003    15568
3     200000004    22547
4     200000005    65702
...         ...      ...
1109  203000449   173068
1110  203000450  3154475
1111  228001883  1421945
1112  233000001   994730
1113  233000592     6155

[1114 rows x 2 columns]

## (2) 노출 심도
### (2) 1. 도로재비산


- 데이터 불러오기 2020년 1월 ~12월 경기도 도로재비산먼지
- http://www.keco.or.kr/kr/main/index.do 한국환경공단 

In [21]:
비산1=pd.read_excel('data/20210322155612.xlsx')
비산2=pd.read_excel('data/20210322155621.xlsx')
비산3=pd.read_excel('data/20210322155630.xlsx')
비산4=pd.read_excel('data/20210322155643.xlsx')
비산5=pd.read_excel('data/20210322155648.xlsx')
비산6=pd.read_excel('data/20210322155653.xlsx')
비산7=pd.read_excel('data/20210322155701.xlsx')
비산8=pd.read_excel('data/20210322155704.xlsx')
비산9=pd.read_excel('data/20210322155708.xlsx')
비산10=pd.read_excel('data/20210322155713.xlsx')
비산11=pd.read_excel('data/20210322155718.xlsx')
비산12=pd.read_excel('data/20210322155723.xlsx')
비산13=pd.read_excel('data/20210322155726.xlsx')
비산14=pd.read_excel('data/20210322155728.xlsx')

In [22]:
roaddust=[비산1,비산2,비산3,비산4,비산5,비산6,
         비산7,비산8,비산9,비산10,비산11,비산12]
roaddust[0].sample(5)

측정일자   측정시간 시도명  시군구명           도로명                    도로명주소  \
2   2020-01-03  14:56  경기  김포시   김포한강3로 (서→동)  경기도 김포시 양촌읍 구래리 823-14답   
8   2020-01-03      -  경기  김포시   김포한강9로 (남→북)        경기도 김포시 구래동 6900도   
11  2020-01-03      -  경기  김포시      양곡로 (서→동)  경기도 김포시 양촌읍 양곡리 371-25도   
3   2020-01-03  15:17  경기  김포시   김포한강2로 (동→서)      경기도 김포시 운양동 1406-3도   
7   2020-01-03      -  경기  김포시   김포한강8로 (북→남)        경기도 김포시 구래동 6900도   

    재비산먼지평균농도(㎍/㎥)  구간상태  
2            126.0    보통  
8            512.6  매우나쁨  
11           218.9  매우나쁨  
3             42.0  매우좋음  
7            591.9  매우나쁨

- 수원시 데이터만 뽑아내기

In [23]:
ans=[]
for i in roaddust:
    idx=[j.startswith('수원시') for j in i.시군구명]
    i=i[idx]
    if len(i)!=0:
        ans.append(i)
ans[0].sample(5)    

측정일자   측정시간 시도명     시군구명           도로명                     도로명주소  \
347  2020-03-23  10:10  경기  수원시 권선구    서수원로 (북→남)    경기도 수원시 권선구 당수동 산25-1임   
389  2020-03-24  12:39  경기  수원시 영통구  센트럴타운로 (동→서)  경기도 수원시 영통구 이의동 1381-23도   
380  2020-03-24  11:24  경기  수원시 팔달구     효원로 (서→동)    경기도 수원시 팔달구 매산로3가 5-6도   
378  2020-03-24  11:05  경기  수원시 권선구     수성로 (동→서)    경기도 수원시 권선구 서둔동 279-1도   
375  2020-03-24  10:47  경기  수원시 팔달구    월드컵로 (남→북)    경기도 수원시 팔달구 우만동 산77-8도   

     재비산먼지평균농도(㎍/㎥)  구간상태  
347             150    보통  
389              56    좋음  
380              38  매우좋음  
378              15  매우좋음  
375              16  매우좋음

- 필요한 columns만 추출

In [24]:
ans=[i.loc[:,['측정일자','측정시간','도로명','도로명주소','재비산먼지평균농도(㎍/㎥)']] for i in ans]
ans[0]

측정일자   측정시간               도로명                     도로명주소  \
157  2020-03-16  10:41        경수대로 (북→남)      경기도 수원시 장안구 파장동 산45임   
158  2020-03-16  10:57        장다리로 (북→남)      경기도 수원시 팔달구 인계동 930천   
159  2020-03-16  11:02        장다리로 (북→남)     경기도 수원시 권선구 권선동 1069구   
160  2020-03-16  11:10  덕영대로1190번길 (북→남)     경기도 수원시 권선구 권선동 1212도   
161  2020-03-16  11:14  덕영대로1246번길 (남→북)  경기도 수원시 권선구 권선동 1337-9 도   
..          ...    ...               ...                       ...   
390  2020-03-24  12:48      센트럴파크로 (북→남)   경기도 수원시 영통구 이의동 1379-5도   
391  2020-03-24  13:40       광교호수로 (북→남)      경기도 수원시 영통구 하동 72-1도   
392  2020-03-24  13:51       에듀타운로 (동→서)  경기도 수원시 영통구 이의동 1381-22도   
393  2020-03-24  13:59         대학로 (남→북)   경기도 수원시 영통구 이의동 912-12도   
394  2020-03-24  14:04       웰빙타운로 (서→동)   경기도 수원시 영통구 이의동 1373-1도   

     재비산먼지평균농도(㎍/㎥)  
157               9  
158              96  
159             125  
160              14  
161              40  
..              ...  
390               9  
391              11  
392               9  
393              11  
394              31  

[76 rows x 5 columns]

### 비산먼지는 정기적으로 측정하는 것이 아님
### 따라서 도로재비산먼지를 인근 도시측정소 농도로 나눠줘서 도로재비산의 영향력만을 살펴봄
* 비산먼지농도에 반영된 날씨, 시계열적 특성 등 외부요인 통제해야 수치비교 가능
* 비산먼지 측정일과 측정시간에 해당하는 주변 도시 측정소 미세먼지 농도를 활용
* 도로재비산먼지/도시측정소농도 
* 도로와 차량여건 이외의=> 도로재비산먼지에 영향을 미치는 외부요인 통제(날씨, 추세 등)

- 측정일자와 시간을 합친 열을 생성하고 날짜 데이터로 변환

In [25]:
for i in ans:
    i['측정']=i.측정일자+' '+i.측정시간

for j in ans:
    j.측정=[dt.datetime.strptime(i, "%Y-%m-%d %H:%M") for i in j.측정]

### 경기도 2020년 도시대기측정데이터
#### 1. compas 제공 데이터는 월 평균이라 사용하기에 부적절
#### 2. 경기도 공공데이터드림에서 경기도 2020년 데이터를 가져와서 수원시만 뽑아냄
#### 3. 데이터가 이상하게 encoding이 cp949와 utf-8이 섞여있음.

- https://data.gg.go.kr/portal/mainPage.do  (경기도 데이터드림)
- 2020년 대기질 측정정보 가져오기

In [26]:
front='data/대기질측정정보_2020년'
end='월.csv'
data=[]
for i in range(1,13):
    filename=front+str(i).zfill(2)+end
    try:        
        data.append(pd.read_csv(filename, encoding='cp949'))
    except:
        data.append(pd.read_csv(filename, encoding='utf-8'))

- 수원시 추려내고, 도시대기만 뽑아내기

In [27]:
misaedust_Suwoon=pd.DataFrame()
for j in range(0,12):
    misaedust_Suwoon=misaedust_Suwoon.append(data[j].loc[(data[j].시군명=='수원시')&(data[j].측정망명!='도로변대기'),:])
misaedust_Suwoon.sample(5)   

시군코드  시군명 측정소명    설치년도  측정망명             측정일시각  아황산가스농도값  일산화탄소농도값  \
32404  41110  수원시  인계동  1987.0  도시대기  2020-10-15 22:00     0.003       0.6   
30240  41110  수원시  신풍동  1986.0  도시대기  2020-02-05 18:00     0.002       0.4   
20854  41110  수원시  광교동  1994.0  도시대기  2020-10-10 16:00     0.002       0.3   
16250  41110  수원시  천천동  2003.0  도시대기  2020-07-14 16:00       NaN       0.2   
21002  41110  수원시  인계동  1987.0  도시대기  2020-12-12 07:00     0.005       0.8   

       오존농도값  이산화질소농도값  미세먼지PM10농도값  미세먼지PM2.5농도값  
32404  0.001     0.054         37.0           7.0  
30240  0.016     0.020         31.0          13.0  
20854  0.053     0.007         22.0           4.0  
16250  0.027     0.009          6.0           1.0  
21002  0.008     0.050         52.0          31.0

- 날짜 데이터로 바꾸기 위해서 24:00 오류 발생시키는 데이터 제거

In [28]:
bool_idx=list(map(lambda x: not x.endswith('24:00'),misaedust_Suwoon['측정일시각']))
misaedust_Suwoon=misaedust_Suwoon[bool_idx]
misaedust_Suwoon['측정일시각']=[dt.datetime.strptime(i,"%Y-%m-%d %H:%M") for i in misaedust_Suwoon['측정일시각']]

misaedust_Suwoon.dtypes

시군코드                     int64
시군명                     object
측정소명                    object
설치년도                   float64
측정망명                    object
측정일시각           datetime64[ns]
아황산가스농도값               float64
일산화탄소농도값               float64
오존농도값                  float64
이산화질소농도값               float64
미세먼지PM10농도값            float64
미세먼지PM2.5농도값           float64
dtype: object

1. 도로재비산 하나로 합치기
2. 측정 시간이 분 단위라서 도시미세먼지 측정 데이터와 일치하지 않음
3. 분 단위는 버리고 월.일.시간으로 매치시키기
4. 도시측정소 데이터에 생각보다 결측값이 많아서 월,일로 매치시킨 후 전 후 3시간 오차까지 허용.

In [29]:
roadDust=pd.DataFrame()
for i in ans:
    roadDust=roadDust.append(i)
roadDust.sample(5)

측정일자   측정시간         도로명                      도로명주소  재비산먼지평균농도(㎍/㎥)  \
355  2020-03-23  11:28  오목천로 (서→동)  경기도 수원시 권선구 오목천동 428-10 목              26   
237  2020-11-06  13:59   송정로 (동→서)       경기도 수원시 장안구 조원동 792도              11   
323  2020-11-10  10:39   수성로 (동→서)    경기도 수원시 장안구 조원동 533-11도              52   
557  2020-04-29  11:19   효원로 (서→동)     경기도 수원시 팔달구 매산로3가 5-6도              88   
179  2020-11-05  12:24   반달로 (북→남)      경기도 수원시 영통구 영통동 1055도              68   

                     측정  
355 2020-03-23 11:28:00  
237 2020-11-06 13:59:00  
323 2020-11-10 10:39:00  
557 2020-04-29 11:19:00  
179 2020-11-05 12:24:00

- 월일시 열 만들기
- 도로재비산 측정 월일시 뽑기

In [30]:
misaedust_Suwoon['월일시']=[i.strftime('%m%d%H') for i in misaedust_Suwoon.측정일시각]
roadDust['월일시']=[i.strftime('%m%d%H') for i in roadDust.측정]

target_date=roadDust['월일시'].unique()
target_date

array(['031610', '031611', '031612', '031613', '031614', '032010',
       '032011', '032012', '032013', '032014', '032310', '032311',
       '032313', '032410', '032411', '032412', '032413', '032414',
       '042410', '042411', '042412', '042413', '042710', '042711',
       '042712', '042713', '042714', '042810', '042811', '042812',
       '042813', '042910', '042911', '042912', '042913', '061210',
       '061211', '061212', '061213', '061811', '061812', '061813',
       '061814', '062310', '062311', '062312', '062313', '062910',
       '062911', '062912', '062913', '082410', '082411', '082412',
       '082413', '082510', '082511', '082512', '082513', '082514',
       '082610', '082611', '082612', '083110', '083111', '083112',
       '083113', '100610', '100611', '100612', '100710', '100711',
       '100712', '100713', '101210', '101211', '101212', '101310',
       '101311', '101312', '101313', '110510', '110511', '110512',
       '110513', '110610', '110611', '110612', '110613', '1109

- 도시측정소 데이터 중 도로재비산 월일시에 해당되는 애들만 뽑기

In [31]:
bool_idx= [i in target_date for i in misaedust_Suwoon['월일시']]
target_misae=misaedust_Suwoon[bool_idx]
target_misae.sample(5)

시군코드  시군명  측정소명    설치년도  측정망명               측정일시각  아황산가스농도값  일산화탄소농도값  \
4043   41110  수원시   고색동  2006.0  도시대기 2020-12-04 13:00:00     0.004       0.5   
20135  41110  수원시  호매실동     NaN  도시대기 2020-06-12 11:00:00     0.004       0.6   
1082   41110  수원시   인계동  1987.0  도시대기 2020-08-31 11:00:00     0.003       0.6   
12565  41110  수원시   인계동  1987.0  도시대기 2020-03-23 13:00:00     0.005       0.5   
16201  41110  수원시   신풍동  1986.0  도시대기 2020-08-24 11:00:00     0.002       0.4   

       오존농도값  이산화질소농도값  미세먼지PM10농도값  미세먼지PM2.5농도값     월일시  
4043   0.018     0.028         49.0          23.0  120413  
20135  0.019     0.028         44.0          21.0  061211  
1082   0.017     0.014         11.0           1.0  083111  
12565  0.039     0.015         46.0          28.0  032313  
16201  0.045     0.012         26.0          12.0  082411

### 도로재비산 측정 지점과 가장 가까운 도시 측정소 찾기
* 1. 도시 측정소별 주소 join하기(외부데이터 경기공공데이터드림)
* 2. 도로재비산 측정지점, 측정소별 주소 위경도로 바꾸기

측정소 위치를 파악하기 위해 주소 데이터 결합하기
- https://data.gg.go.kr/portal/mainPage.do

In [32]:
address=pd.read_excel('data/경기측정소.xlsx')
address.head()

Unnamed: 0 Unnamed: 1                          Unnamed: 2  Unnamed: 3  \
0    측정소 리스트        NaN                                 NaN         NaN   
1        NaN        NaN                                 NaN         NaN   
2        지역명       측정소명                              측정소 주소        운영기관   
3         경기        가남읍  경기도 여주시 가남읍 태평중앙1길 20 가남읍행정복지센터 옥상  경기도보건환경연구원   
4         경기         가평        경기도 가평군 가평읍 석봉로 181 가평군청 의회동  경기도보건환경연구원   

  Unnamed: 4  
0        NaN  
1        NaN  
2       설치년도  
3       2020  
4       2010

- 열 선택

In [33]:
address.columns=address.loc[2,:]
address=address.loc[3:,['측정소명','측정소 주소']]
address.head(5)

2 측정소명                               측정소 주소
3  가남읍   경기도 여주시 가남읍 태평중앙1길 20 가남읍행정복지센터 옥상
4   가평         경기도 가평군 가평읍 석봉로 181 가평군청 의회동
5  경안동                 경기 광주시 중앙로 128 농협중앙회
6  고덕면  경기도 평택시 고덕면 고덕국제2로 111 종덕초등학교 2층 옥상
7  고색동     경기 수원시 권선구 서부로 1600 수원시도로교통관리사업소

- 수원시 측정소만 선택, 주소 결합

In [34]:
target_misae=pd.merge(target_misae,address,how='inner',on='측정소명')
target_misae.sample(2)

시군코드  시군명 측정소명    설치년도  측정망명               측정일시각  아황산가스농도값  일산화탄소농도값  \
2    41110  수원시  고색동  2006.0  도시대기 2020-03-24 12:00:00     0.004       0.6   
140  41110  수원시  광교동  1994.0  도시대기 2020-10-13 13:00:00     0.002       0.4   

     오존농도값  이산화질소농도값  미세먼지PM10농도값  미세먼지PM2.5농도값     월일시  \
2    0.034     0.031         68.0          55.0  032412   
140  0.033     0.013         30.0           7.0  101313   

                               측정소 주소  
2    경기 수원시 권선구 서부로 1600 수원시도로교통관리사업소  
140          경기 수원시 영통구 법조로 129 이의중학교

가까운 측정소 매칭( 도로명 주소 -> 위경도 변환)

### 네이버 API 지오코딩

In [35]:
def geocoding_NaverApi(addr):
    ClientID='azxbhbvbvs' #네이버클라우드 발급 ID
    ClientSecret='2aCnmWXiSpF4V1mbFIcM623lIT0elhGNZugOZlam' # 발급key
    query_content=urllib.parse.quote(addr)
    url='https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode?query='+query_content #요청 url base
    request= urllib.request.Request(url)
    request.add_header('X-NCP-APIGW-API-KEY-ID', ClientID)
    request.add_header('X-NCP-APIGW-API-KEY', ClientSecret)
    response = urllib.request.urlopen(request)
    #요청 상태 확인
    rescode = response.getcode()
    if(rescode==200): #정상이면
        response_body = response.read()
        contents=json.loads(response_body.decode('utf-8'))['addresses'][0] #utf-8로 디코딩, json 형태 addresses 속성 값 추출
        return float(contents['x']),float(contents['y']) #위경도 추출
    else:
        print("Error Code:" + rescode)

#### 변환할 주소 선정
    1. 도시측정소
    2. 도로재비산 측정위치

- 변환 주소 추출

In [36]:
addr_misae=target_misae['측정소 주소'].unique()
addr_roadDust=roadDust['도로명주소'].unique()

- 위경도 추출, 주소: 위경도 dict 구조로 변수에 저장

In [37]:
lonlat_misae={i:geocoding_NaverApi(i,) for i in addr_misae}
lonlat_roadDust={i:geocoding_NaverApi(i) for i in addr_roadDust}

- 위경도 값만 뽑아서 임시 변수에 저장

In [38]:
temp1=np.array(list(lonlat_misae.values()))
temp2=np.array(list(lonlat_roadDust.values()))

- 측정소 주소에 대응되는 측정소 이름

In [39]:
trans_dict={}

for i in range(len(addr_misae)):
    trans_dict[addr_misae[i]]=target_misae.측정소명[target_misae['측정소 주소']==addr_misae[i]].unique()[0]
trans_dict

{'경기 수원시 권선구 서부로 1600 수원시도로교통관리사업소': '고색동',
 '경기 수원시 영통구 법조로 129 이의중학교': '광교동',
 '경기 수원시 팔달구 신풍로 23번길 68 선경도서관': '신풍동',
 '경기 수원시 영통구 영통로 217번길 12 영통2동 행정복지센터': '영통동',
 '경기 수원시 팔달구 효원로 241 수원시청': '인계동',
 '경기 수원시 장안구 서부로 2066 성균관대학교 제2공학관': '천천동',
 '경기도 수원시 권선구 칠보로 161 호매실동 행정복지센터 (호매실동)': '호매실동'}

- 도로 주소에 대응되는 측정소 이름(가장 가까운 도시 측정소)
- 위경도로 가장 가까운 주소를 찾고, 주소 측정소 대응표를 거쳐서 측정소에 매칭
- 유클리디안 거리 이용

In [40]:
trans_dict2={}
for i in range(len(temp2)):
    temp3=np.sqrt(np.apply_along_axis(sum,1,(temp1-temp2[i])**2))
    trans_dict2[addr_roadDust[i]]=trans_dict[list(lonlat_misae.keys())[np.where(temp3.min()==temp3)[0][0]]]
    
#인근 측정소 찾기
roadDust['인근측정소']=[trans_dict2[i] for i in roadDust['도로명주소']]

### 도로재비산 측정일과 매치된 도시 측정소 간 측정일 매칭하기
- 월일시 매치 안되는 경우 -> 월일로 우선 매치

In [41]:
misaedust_Suwoon['월일']=[i.strftime('%m%d') for i in misaedust_Suwoon.측정일시각]
roadDust['월일']=[i.strftime('%m%d') for i in roadDust.측정]

In [42]:
misaedust_Suwoon['월일'][:3]

132    0131
133    0131
134    0131
Name: 월일, dtype: object

- 도로 측정일 월일에 포함되는 도시 측정소 농도 데이터를 target_misae로 설정

In [43]:
target_date=roadDust['월일'].unique()

bool_idx= [i in target_date for i in misaedust_Suwoon['월일']]
target_misae=misaedust_Suwoon[bool_idx]
target_misae.sample(5)

시군코드  시군명 측정소명    설치년도  측정망명               측정일시각  아황산가스농도값  일산화탄소농도값  \
18863  41110  수원시  인계동  1987.0  도시대기 2020-03-20 17:00:00     0.003       0.5   
9167   41110  수원시  영통동  2001.0  도시대기 2020-06-23 06:00:00     0.003       0.6   
9556   41110  수원시  영통동  2001.0  도시대기 2020-03-24 21:00:00     0.004       0.5   
25171  41110  수원시  신풍동  1986.0  도시대기 2020-10-12 15:00:00     0.003       0.5   
13934  41110  수원시  영통동  2001.0  도시대기 2020-08-25 11:00:00     0.003       0.4   

       오존농도값  이산화질소농도값  미세먼지PM10농도값  미세먼지PM2.5농도값     월일시    월일  
18863  0.034     0.010         32.0           6.0  032017  0320  
9167   0.048     0.013         52.0          37.0  062306  0623  
9556   0.030     0.036         35.0          27.0  032421  0324  
25171  0.065     0.010         50.0          13.0  101215  1012  
13934  0.038     0.011         26.0          23.0  082511  0825

In [44]:
len(target_misae)

3576

In [45]:
roadDust.head(3)

측정일자   측정시간         도로명                  도로명주소  재비산먼지평균농도(㎍/㎥)  \
157  2020-03-16  10:41  경수대로 (북→남)   경기도 수원시 장안구 파장동 산45임               9   
158  2020-03-16  10:57  장다리로 (북→남)   경기도 수원시 팔달구 인계동 930천              96   
159  2020-03-16  11:02  장다리로 (북→남)  경기도 수원시 권선구 권선동 1069구             125   

                     측정     월일시 인근측정소    월일  
157 2020-03-16 10:41:00  031610   천천동  0316  
158 2020-03-16 10:57:00  031610   인계동  0316  
159 2020-03-16 11:02:00  031611   인계동  0316

- 월일로 매치하기

In [46]:
final_roadDust=pd.merge(target_misae,roadDust,how='left',
        left_on=['월일','측정소명'],right_on=['월일','인근측정소'])
final_roadDust.head(3)

시군코드  시군명 측정소명    설치년도  측정망명               측정일시각  아황산가스농도값  일산화탄소농도값  \
0  41110  수원시  고색동  2006.0  도시대기 2020-03-24 23:00:00     0.003       0.6   
1  41110  수원시  광교동  1994.0  도시대기 2020-03-24 23:00:00     0.005       0.6   
2  41110  수원시  광교동  1994.0  도시대기 2020-03-24 23:00:00     0.005       0.6   

   오존농도값  이산화질소농도값  ...   월일시_x    월일        측정일자   측정시간          도로명  \
0  0.016     0.040  ...  032423  0324         NaN    NaN          NaN   
1  0.003     0.064  ...  032423  0324  2020-03-24  10:39   월드컵로 (동→서)   
2  0.003     0.064  ...  032423  0324  2020-03-24  12:25  광교중앙로 (남→북)   

                     도로명주소 재비산먼지평균농도(㎍/㎥)                  측정   월일시_y 인근측정소  
0                      NaN            NaN                 NaT     NaN   NaN  
1     경기도 수원시 영통구 원천동 626도           30.0 2020-03-24 10:39:00  032410   광교동  
2  경기도 수원시 영통구 원천동 620-14도           20.0 2020-03-24 12:25:00  032412   광교동  

[3 rows x 22 columns]

- 월일로 매치가 안되는 데이터는 제거

In [47]:
final_roadDust=final_roadDust[final_roadDust['인근측정소'].notnull()]
final_roadDust.head(3)

시군코드  시군명 측정소명    설치년도  측정망명               측정일시각  아황산가스농도값  일산화탄소농도값  \
1  41110  수원시  광교동  1994.0  도시대기 2020-03-24 23:00:00     0.005       0.6   
2  41110  수원시  광교동  1994.0  도시대기 2020-03-24 23:00:00     0.005       0.6   
3  41110  수원시  광교동  1994.0  도시대기 2020-03-24 23:00:00     0.005       0.6   

   오존농도값  이산화질소농도값  ...   월일시_x    월일        측정일자   측정시간          도로명  \
1  0.003     0.064  ...  032423  0324  2020-03-24  10:39   월드컵로 (동→서)   
2  0.003     0.064  ...  032423  0324  2020-03-24  12:25  광교중앙로 (남→북)   
3  0.003     0.064  ...  032423  0324  2020-03-24  12:35    법조로 (동→서)   

                     도로명주소 재비산먼지평균농도(㎍/㎥)                  측정   월일시_y 인근측정소  
1     경기도 수원시 영통구 원천동 626도           30.0 2020-03-24 10:39:00  032410   광교동  
2  경기도 수원시 영통구 원천동 620-14도           20.0 2020-03-24 12:25:00  032412   광교동  
3      경기도 용인시수지구 상현동 916도           19.0 2020-03-24 12:35:00  032412   광교동  

[3 rows x 22 columns]

### 도시측정소 측정일과 도로측정소 측정일 GAP 구하기
- 3시간 이내까지만 허용, 그 이후에는 날씨 및 차랑 통행량 등에 영향을 받을 수 있기 때문에 제외

In [48]:
final_roadDust['월일시_x']=[int(i) for i in final_roadDust['월일시_x']]
final_roadDust['월일시_y']=[int(i) for i in final_roadDust['월일시_y']]

final_roadDust['date_gap']=abs(final_roadDust['월일시_x']-final_roadDust['월일시_y'])

final_roadDust.dtypes

시군코드                       int64
시군명                       object
측정소명                      object
설치년도                     float64
측정망명                      object
측정일시각             datetime64[ns]
아황산가스농도값                 float64
일산화탄소농도값                 float64
오존농도값                    float64
이산화질소농도값                 float64
미세먼지PM10농도값              float64
미세먼지PM2.5농도값             float64
월일시_x                      int64
월일                        object
측정일자                      object
측정시간                      object
도로명                       object
도로명주소                     object
재비산먼지평균농도(㎍/㎥)           float64
측정                datetime64[ns]
월일시_y                      int64
인근측정소                     object
date_gap                   int64
dtype: object

- 도시측정소 데이터와 도로재비산 측정 데이터 측정시간 차이 3시간 이내에 해당하는 것들 간추리기

In [49]:
final_roadDust3=final_roadDust[final_roadDust['date_gap']<=3]

-  도로재비산농도/도시측정농도: 로 재비산 심각정도 파악

In [50]:
final_roadDust3['도로재비산심각도']=final_roadDust3['재비산먼지평균농도(㎍/㎥)']/final_roadDust3['미세먼지PM10농도값']

/opt/app-root/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


- 도로재비산 측정 지점별 연평균 재비산심각도

In [51]:
grouped=final_roadDust3.groupby(by=['도로명주소'])
final_roadDust3mean=pd.DataFrame(grouped.mean().도로재비산심각도).sort_values(by='도로재비산심각도',ascending=False)

###  최종데이터 열 정리 및 정보 추가
- 최종 데이터에 위경도 정보 추가

In [52]:
final_roadDust3mean['경도']=[lonlat_roadDust[i][0] for i in final_roadDust3mean.index]
final_roadDust3mean['위도']=[lonlat_roadDust[i][1] for i in final_roadDust3mean.index]
final_roadDust3mean.head(5)

도로재비산심각도          경도         위도
도로명주소                                                     
경기도 수원시 장안구 영화동 107-7도    11.229169  127.019560  37.292726
경기도 수원시 팔달구 화서동 360-1도     9.001188  126.992249  37.286912
경기도 수원시 영통구 매탄동 950-1도     6.157271  127.036793  37.266257
경기도 수원시 권선구 세류동 1033-3도    3.703058  127.013975  37.262795
경기도 수원시 권선구 호매실동 36-11 답   3.504247  126.961310  37.269813

- 행이름 변경

In [53]:
final_roadDust3mean['도로명주소']=final_roadDust3mean.index
final_roadDust3mean.index=range(len(final_roadDust3mean))
final_roadDust3mean

도로재비산심각도          경도         위도                     도로명주소
0   11.229169  127.019560  37.292726    경기도 수원시 장안구 영화동 107-7도
1    9.001188  126.992249  37.286912    경기도 수원시 팔달구 화서동 360-1도
2    6.157271  127.036793  37.266257    경기도 수원시 영통구 매탄동 950-1도
3    3.703058  127.013975  37.262795   경기도 수원시 권선구 세류동 1033-3도
4    3.504247  126.961310  37.269813  경기도 수원시 권선구 호매실동 36-11 답
..        ...         ...        ...                       ...
70   0.487810  126.983642  37.233788  경기도 수원시 권선구 오목천동 756-1 도
71   0.421931  127.040784  37.296845   경기도 수원시 영통구 이의동 912-12도
72   0.372182  127.056583  37.234333   경기도 수원시 영통구 망포동 574-6 도
73   0.362797  127.078999  37.293059      경기도 수원시 영통구 하동 72-1도
74   0.355506  126.959238  37.270403  경기도 수원시 권선구 호매실동 1323-3도

[75 rows x 4 columns]

- 도로명 주소, 도로명 JOIN하기 위한 표 만들기

In [54]:
roadnames=roadDust[['도로명주소','도로명']].drop_duplicates(['도로명주소'], keep='first')
roadnames

도로명주소               도로명
157      경기도 수원시 장안구 파장동 산45임        경수대로 (북→남)
158      경기도 수원시 팔달구 인계동 930천        장다리로 (북→남)
159     경기도 수원시 권선구 권선동 1069구        장다리로 (북→남)
160     경기도 수원시 권선구 권선동 1212도  덕영대로1190번길 (북→남)
161  경기도 수원시 권선구 권선동 1337-9 도  덕영대로1246번길 (남→북)
..                        ...               ...
392  경기도 수원시 영통구 이의동 1381-22도       에듀타운로 (동→서)
393   경기도 수원시 영통구 이의동 912-12도         대학로 (남→북)
394   경기도 수원시 영통구 이의동 1373-1도       웰빙타운로 (서→동)
512  경기도 수원시 장안구 연무동 161-13 도        경수대로 (남→북)
515  경기도 수원시 권선구 오목천동 756-1 도        서수원로 (남→북)

[75 rows x 2 columns]

- 도로명 추가하고 CSV파일로 저장

In [55]:
final=pd.merge(roadnames,final_roadDust3mean,how='left',on='도로명주소')
final.to_csv('data/도로재비산심각도.csv',index=False)

## (2) 2. 공장

#### 경기도 대기오염배출 공장 데이터 탐색
[https://data.gg.go.kr/portal/data/service/selectServicePage.do?page=1&sortColumn=&sortDirection=&infId=794V0223GQQ9O1P4WAP221637079&infSeq=1&searchWord=%EA%B3%B5%EC%9E%A5%EB%93%B1%EB%A1%9D']
       경기도 데이터드림 공장등록현황 2020.08

In [56]:
factory=pd.read_excel('data/공장등록현황.xlsx')
factory.head(2)

회사명             행정기관명          단지명   설립구분명     용지면적     건축면적  종업원수  \
0   (주)재능인쇄  (사)교하문발지방산업단지협의회  파주문발2지방산업단지  일반산업단지  16135.6  12840.1  63.0   
1  (주)더원프린팅  (사)교하문발지방산업단지협의회  파주문발1지방산업단지  일반산업단지      0.0   1150.0  10.0   

  공장규모구분명                공장등록일 용도지역명  ... 산업용수사용량(톤/일) 자기자본액(백만) 타인자본액(백만)  \
0     중기업  2001-07-19 00:00:00  도시지역  ...         10.0       0.0       0.0   
1     소기업  2003-09-20 00:00:00  도시지역  ...          0.0       0.0       0.0   

           전화번호 홈페이지주소  소재지우편번호                         소재지도로명주소  \
0  031-956-3122    NaN  10878.0  경기도 파주시 산업단지길 122 (다율동,JEI재능인쇄)   
1  031-949-7072    NaN  10878.0          경기도 파주시 산업단지길 163 (문발동)   

                     소재지지번주소    WGS84위도     WGS84경도  
0  경기도 파주시 다율동 970번지 JEI재능인쇄  37.730311  126.712244  
1          경기도 파주시 문발동 470번지  37.729844  126.709183  

[2 rows x 28 columns]

In [57]:
factory.columns

Index(['회사명', '행정기관명', '단지명', '설립구분명', '용지면적', '건축면적', '종업원수', '공장규모구분명',
       '공장등록일', '용도지역명', '지목명', '업종명', '업종코드', '생산품정보', '대기오염등급', '수질오염등급',
       '소음진동여부', '생활용수사용량(톤/일)', '산업용수사용량(톤/일)', '자기자본액(백만)', '타인자본액(백만)',
       '전화번호', '홈페이지주소', '소재지우편번호', '소재지도로명주소', '소재지지번주소', 'WGS84위도',
       'WGS84경도'],
      dtype='object')

### 대기오염등급 살펴보기 및 결측치 제거(전처리)

In [58]:
factory.대기오염등급.unique()

array(['해당없음', '5종', '4종', '2종', '1종', '3종', nan], dtype=object)

- 필요 데이터 선택, 대기오염 등급 결측치 제거

In [59]:
airfactory=factory[[i in ['5종', '4종', '2종', '1종', '3종'] for i in factory.대기오염등급]]
airfactory=airfactory[['용지면적', '건축면적', '공장규모구분명', '대기오염등급', 'WGS84위도', 'WGS84경도', '소재지도로명주소']]
airfactory.shape

(5119, 7)

In [60]:
airfactory.columns

Index(['용지면적', '건축면적', '공장규모구분명', '대기오염등급', 'WGS84위도', 'WGS84경도', '소재지도로명주소'], dtype='object')

- dataframe -> geodataframe 변경 후 버스정류소와의 거리 살펴보기

In [61]:
air_fact=gpd.points_from_xy(airfactory.WGS84경도,airfactory.WGS84위도)
cols=['용지면적', '건축면적', '공장규모구분명', '대기오염등급','소재지도로명주소']
gdf_fact=gpd.GeoDataFrame(airfactory[cols], geometry=air_fact,
                         crs=from_epsg(4326))
gdf_fact.index=range(len(gdf_fact))
gdf_fact.tail(5)

/opt/app-root/lib/python3.6/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


용지면적      건축면적 공장규모구분명 대기오염등급  \
5114   3310.0   3484.68     소기업     5종   
5115  89090.3  43644.85     대기업     5종   
5116  55556.0  17485.60     소기업     5종   
5117  62391.3   2603.60     중기업     5종   
5118  62391.3   2603.60     중기업     5종   

                                               소재지도로명주소  \
5114         경기도 안산시 단원구 엠티브이5로 55 시화MTV 7사 102호] (성곡동)   
5115           경기도 안산시 단원구 시화벤처로 575 시화MTV 5사 102 (성곡동)   
5116  경기도 안산시 단원구 엠티브이1로 75 (목내동 513, 시화MTV 7사 309) ...   
5117                     경기도 평택시 포승읍 서동대로 437-129 (신영리)   
5118                     경기도 평택시 포승읍 서동대로 437-129 (신영리)   

                        geometry  
5114  POINT (126.76322 37.29990)  
5115  POINT (126.73726 37.30637)  
5116  POINT (126.76463 37.29870)  
5117  POINT (126.87452 36.93788)  
5118  POINT (126.87452 36.93788)

- 위경도 추출

In [62]:
fact_latlon=np.array([[gdf_fact['geometry'][0].x,gdf_fact['geometry'][0].y] for i in gdf_fact.geometry])
fact_latlon[:3]

array([[126.7119601,  37.7276165],
       [126.7119601,  37.7276165],
       [126.7119601,  37.7276165]])

### 등급별 대기오염 배출 정보: 환경부
* 1종 : 연간 오염물질배출량이 80톤 이상이거나 1일 평균 폐수배출량 2,000㎥이상,
* 2종 : 연간 오염물질배출량이 20 이상 80톤 미만이거나, 1일평균 폐수배출량 700㎥이상 2,000㎥미만,
* 3종 : 연간 오염물질배출량이 10톤 이상 20톤 미만이거나 1일평균 폐수배출량 200㎥이상 700㎥미만,
* 4종 : 연간 오염물질배출량이 2톤이상 10톤미만이거나 1일평균 폐수배출량 50㎥이상 200㎥미만,
* 5종 : 그외 기타 배출시설 또는 소음진동 배출시설 사업장, 폐수배출량에는 용수상용량을 기준한다.

- 버스정류장(수원시) 위경도

In [63]:
busStop=pd.read_csv('1.수원시_버스정류장.csv')
busStop=busStop[busStop.BIS설치여부==1.0]
busStop.sample(4)

정류장ID                정류장명  정류장유형 환승역타입         위치(동)   쉘터  BIS설치여부  \
548  200000047               조원뉴타운     시내    일반  경기도 수원시 조원1동  1.0      1.0   
412  201000122  고색동태산아파트.도로교통관리사업소  시내,마을    일반    경기도 수원시 평동  1.0      1.0   
759  203000285        광교휴먼시아32단지정문     시내    일반  경기도 수원시 광교1동  1.0      1.0   
616  202000008              동수원사거리  시내,마을    일반   경기도 수원시 인계동  1.0      1.0   

     LED  LCD  LED+LCD복합형  알뜰형  임대형(음성)         lon        lat  
548  1.0  NaN         NaN  NaN      NaN  127.018224  37.304523  
412  1.0  NaN         NaN  NaN      NaN  126.975892  37.250616  
759  NaN  1.0         NaN  NaN      NaN  127.067750  37.292814  
616  1.0  NaN         NaN  NaN      NaN  127.029392  37.278353

In [64]:
busStop_target=busStop[['정류장ID','위치(동)','환승역타입','lon', 'lat']]

In [65]:
temp1=np.apply_along_axis(sum,1,abs(fact_latlon-np.array(busStop_target[['lon','lat']])[0]))
temp1

array([0.72965602, 0.72965602, 0.72965602, ..., 0.72965602, 0.72965602,
       0.72965602])

- 버스정류장 데이터 gpd로 변환

In [66]:
bus_gpd_geom=gpd.points_from_xy(busStop_target.lon,busStop_target.lat)
cols=['정류장ID', '위치(동)', '환승역타입']
bus_gpd=gpd.GeoDataFrame(busStop_target[cols], geometry=bus_gpd_geom,
                         crs=from_epsg(4326))
bus_gpd.tail(5)

/opt/app-root/lib/python3.6/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


정류장ID         위치(동) 환승역타입                    geometry
1107  202000249  경기도 수원시 화서1동    일반  POINT (126.99970 37.27886)
1108  202000241  경기도 수원시 화서2동    일반  POINT (126.99582 37.28432)
1109  202000246  경기도 수원시 화서2동    일반  POINT (126.98171 37.28799)
1110  202000239  경기도 수원시 화서2동    일반  POINT (126.98608 37.28666)
1111  202000238  경기도 수원시 화서2동    일반  POINT (126.98510 37.28951)

### 버스정류장 반경 2.5km 이내 공장 찾기
#### 논문근거 2.5km까지는 영향 있음

### 커널 밀도 추정을 활용한 공장 대기오염 밀도
#### 참고논문:
* 1. 주성분분석과 커널밀도함수를 이용한 미세먼지 측정소 적지분석
* 2. 산업단지 주변의 PM-10 확산특성을 고려한 주거계획기준 연구

In [67]:
bus_gpd.geometry[0].distance(gdf_fact.geometry[0])

0.5372043196117452

- 거리 m로 표시하기 위해서 crs변경

In [68]:
bus_gpd5179=bus_gpd.to_crs({'init':'epsg:5179'})
gdf_fact5179=gdf_fact.to_crs({'init':'epsg:5179'})

bus_gpd5179.index=range(len(bus_gpd5179))

/opt/app-root/lib/python3.6/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


* 1종 : 연간 오염물질배출량이 80톤 이상이거나 1일 평균 폐수배출량 2,000㎥이상,
* 2종 : 연간 오염물질배출량이 20 이상 80톤 미만이거나, 1일평균 폐수배출량 700㎥이상 2,000㎥미만,
* 3종 : 연간 오염물질배출량이 10톤 이상 20톤 미만이거나 1일평균 폐수배출량 200㎥이상 700㎥미만,
* 4종 : 연간 오염물질배출량이 2톤이상 10톤미만이거나 1일평균 폐수배출량 50㎥이상 200㎥미만,
* 5종 : 2톤 미만, 폐수배출량에는 용수상용량을 기준한다.

In [69]:
import math
#커널밀도함수: (2.5km이내 공장i 배출량)/(공장과의 거리**2)*원주율

dist=[bus_gpd5179.geometry[1].distance(gdf_fact5179.geometry[i]) for i in range(len(gdf_fact5179.geometry))]

#배출량: 배출량 등급 분류기준 중앙값
density_from_fact=[]
degree_to_ton={'1종':150000000,'2종':50000000,'3종':15000000,'4종':6000000,'5종':1500000}
gdf_fact['연배출량']=[degree_to_ton[i] for i in gdf_fact.대기오염등급]


In [70]:
for j in range(len(bus_gpd5179)):
    dist=[bus_gpd5179.geometry[j].distance(gdf_fact5179.geometry[i]) for i in range(len(gdf_fact5179.geometry))]
    idx=[np.array(dist)<=2500]  # 2.5km 이내의 공장 배출량 밀도로 공장 영향력 측정
    density_from_fact.append(sum((np.array(gdf_fact['연배출량']*100)[idx]/((np.array(dist)[idx])**2)*math.pi)))

/opt/app-root/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  after removing the cwd from sys.path.


- 정류소 데이터에 결합 후 저장

In [71]:
bus_gpd['공장오염밀도']=density_from_fact
bus_gpd.to_file('data/버스정류장별공장.geojson',driver='GeoJSON')

- 대기오염배출량 현황 2017-2019
[https://cleansys.or.kr/statAnnual.do#none]

### Folium으로 시각화

In [72]:
soowon_SIGUN=gpd.read_file('29.수원시_법정경계(시군구).geojson')
soowon_DONG=gpd.read_file('30.수원시_법정경계(읍면동).geojson')

In [73]:
bus_gpd_desc=gpd.read_file('data/버스정류장별공장.geojson')

bus_gpd_desc=bus_gpd_desc.sort_values(by='공장오염밀도',ascending=False)
bus_position=[[i.y,i.x] for i in bus_gpd_desc.geometry]

soowon_SIGUN.head(3)

SIG_CD SIG_KOR_NM                                           geometry
0  41111    수원시 장안구  MULTIPOLYGON (((127.01976 37.35104, 127.02008 ...
1  41113    수원시 권선구  MULTIPOLYGON (((127.03723 37.23684, 127.03679 ...
2  41115    수원시 팔달구  MULTIPOLYGON (((127.03582 37.29361, 127.03644 ...

In [74]:
soowon_DONG.head(3)

EMD_CD EMD_KOR_NM                                           geometry
0  41111129        파장동  MULTIPOLYGON (((127.01079 37.31908, 127.01047 ...
1  41111130        정자동  MULTIPOLYGON (((126.98697 37.31147, 126.98705 ...
2  41111131        이목동  MULTIPOLYGON (((126.98105 37.32685, 126.98160 ...

In [75]:
m=folium.Map([37.281796, 127.016808],zoom_start=12)

for i in range(len(bus_position)):
    folium.Circle(
    location=bus_position[i],
    radius=30,
    color='red').add_to(m)
m

In [76]:
bus_gpd_desc.index=range(len(bus_gpd_desc))
bus_gpd_desc.head(5)

정류장ID         위치(동) 환승역타입        공장오염밀도                    geometry
0  203000237  경기도 수원시 매탄3동    일반  1.077954e+06  POINT (127.04950 37.24856)
1  203000240  경기도 수원시 매탄3동    일반  8.400680e+05  POINT (127.04914 37.24857)
2  203000385  경기도 수원시 매탄3동    일반  8.077172e+05  POINT (127.04360 37.25053)
3  203000130   경기도 수원시 태장동    일반  6.788980e+05  POINT (127.04707 37.24822)
4  203000384   경기도 수원시 태장동    일반  6.729817e+05  POINT (127.04335 37.25027)

In [77]:
df_info=bus_gpd_desc.loc[:,['정류장ID','위치(동)','공장오염밀도']]

- 오염도 top50 시각화

In [78]:
m=folium.Map([37.281796, 127.016808],zoom_start=12)

data=bus_position[:50]
popups=[str(i)+': '+str(bus_gpd_desc['공장오염밀도'][i]) for i in range(len(bus_gpd_desc[:50]))]

plugins.MarkerCluster(data,popups=popups).add_to(m)
    
folium.GeoJson(
    soowon_SIGUN,
    name='soowon_municipalities',
).add_to(m)
m

In [79]:
bus_position[:30]

[[37.24855755, 127.049497],
 [37.24857401, 127.0491364],
 [37.25053007, 127.0436037],
 [37.24822105, 127.0470702],
 [37.25026585, 127.0433476],
 [37.26859755, 127.0667216],
 [37.24745962, 127.0475535],
 [37.24686078, 127.0490741],
 [37.2622237, 127.0601999],
 [37.26214072, 127.0598571],
 [37.26522663, 127.06060890000002],
 [37.26671444, 127.05685420000002],
 [37.25904039, 127.06485079999999],
 [37.26489314, 127.0608116],
 [37.26690432, 127.0569575],
 [37.31149436, 126.9780321],
 [37.26667027, 127.0594081],
 [37.25896162, 127.0651322],
 [37.25181516, 127.0521097],
 [37.25196733, 127.0518947],
 [37.25558179, 126.9908037],
 [37.26899009, 127.066595],
 [37.3116101, 126.9776909],
 [37.24540123, 127.051993],
 [37.267215, 127.06746409999998],
 [37.24585968, 127.05344679999999],
 [37.25543763, 126.9908263],
 [37.265594, 127.0627447],
 [37.26742725, 127.0562363],
 [37.26700274, 127.0662357]]

In [80]:
bus_gpd_desc

정류장ID         위치(동) 환승역타입        공장오염밀도                    geometry
0    203000237  경기도 수원시 매탄3동    일반  1.077954e+06  POINT (127.04950 37.24856)
1    203000240  경기도 수원시 매탄3동    일반  8.400680e+05  POINT (127.04914 37.24857)
2    203000385  경기도 수원시 매탄3동    일반  8.077172e+05  POINT (127.04360 37.25053)
3    203000130   경기도 수원시 태장동    일반  6.788980e+05  POINT (127.04707 37.24822)
4    203000384   경기도 수원시 태장동    일반  6.729817e+05  POINT (127.04335 37.25027)
..         ...           ...   ...           ...                         ...
923  203000322  경기도 수원시 광교1동    일반  0.000000e+00  POINT (127.05144 37.29104)
924  201000173   경기도 수원시 구운동    일반  0.000000e+00  POINT (126.97056 37.27427)
925  203000263  경기도 수원시 광교1동    일반  0.000000e+00  POINT (127.04734 37.30386)
926  201000172   경기도 수원시 구운동    일반  0.000000e+00  POINT (126.97464 37.27526)
927  200000246  경기도 수원시 조원2동    일반  0.000000e+00  POINT (127.01474 37.30619)

[928 rows x 5 columns]

In [81]:
m=folium.Map([37.281796, 127.016808],zoom_start=12)

for i in range(len(bus_position[:30])):
    df=pd.DataFrame(bus_gpd_desc.loc[i,'정류장ID':'공장오염밀도'])
    html=df.to_html(classes='table table-striped table-hover table-condensed table-responsive')
    icon_number=plugins.BeautifyIcon(
    number=i,
    border_color='darkred',
    text_color='darkred',
    inner_icon_style='margin-top:0;')
    
    folium.Marker(
    location=bus_position[i],
    popup=html,
    icon=icon_number).add_to(m)
    
    
folium.GeoJson(
    soowon_SIGUN,
    name='soowon_municipalities',
    
).add_to(m)
m

- 오염도 min-max 정규화

In [82]:
max_fact=bus_gpd_desc['공장오염밀도'].max()
min_fact=bus_gpd_desc['공장오염밀도'].min()
bus_gpd_desc['최소최대정규화오염도']=(bus_gpd_desc['공장오염밀도']-min_fact)/(max_fact-min_fact)

In [83]:
bus_gpd_desc.to_file('data/버스정류장별공장.geojson',driver='GeoJSON')

## (1) 3. 교통량, 도로재비산 정류장 연결작업

### 1. 도로재비산심각도와  정류장 연결

In [84]:
dust = pd.read_csv('data/도로재비산심각도.csv')
dust

도로명주소               도로명  도로재비산심각도          경도  \
0       경기도 수원시 장안구 파장동 산45임        경수대로 (북→남)  0.588181  126.984855   
1       경기도 수원시 팔달구 인계동 930천        장다리로 (북→남)  2.863450  127.032065   
2      경기도 수원시 권선구 권선동 1069구        장다리로 (북→남)  1.862622  127.022046   
3      경기도 수원시 권선구 권선동 1212도  덕영대로1190번길 (북→남)  0.840071  127.022257   
4   경기도 수원시 권선구 권선동 1337-9 도  덕영대로1246번길 (남→북)  1.556415  127.021207   
..                       ...               ...       ...         ...   
70  경기도 수원시 영통구 이의동 1381-22도       에듀타운로 (동→서)  1.624199  127.054247   
71   경기도 수원시 영통구 이의동 912-12도         대학로 (남→북)  0.421931  127.040784   
72   경기도 수원시 영통구 이의동 1373-1도       웰빙타운로 (서→동)  0.723686  127.038303   
73  경기도 수원시 장안구 연무동 161-13 도        경수대로 (남→북)  0.826423  127.024504   
74  경기도 수원시 권선구 오목천동 756-1 도        서수원로 (남→북)  0.487810  126.983642   

           위도  
0   37.328292  
1   37.278025  
2   37.260115  
3   37.249073  
4   37.243330  
..        ...  
70  37.283538  
71  37.296845  
72  37.308580  
73  37.290070  
74  37.233788  

[75 rows x 5 columns]

- 이후 거리계산 시 stack을 하기 위해 도로명 데이터에 공백제거

In [85]:
for i in range(len(dust)):
    dust['도로명'][i] =dust['도로명'][i].replace(' ','')
dust

/opt/app-root/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


도로명주소              도로명  도로재비산심각도          경도         위도
0       경기도 수원시 장안구 파장동 산45임        경수대로(북→남)  0.588181  126.984855  37.328292
1       경기도 수원시 팔달구 인계동 930천        장다리로(북→남)  2.863450  127.032065  37.278025
2      경기도 수원시 권선구 권선동 1069구        장다리로(북→남)  1.862622  127.022046  37.260115
3      경기도 수원시 권선구 권선동 1212도  덕영대로1190번길(북→남)  0.840071  127.022257  37.249073
4   경기도 수원시 권선구 권선동 1337-9 도  덕영대로1246번길(남→북)  1.556415  127.021207  37.243330
..                       ...              ...       ...         ...        ...
70  경기도 수원시 영통구 이의동 1381-22도       에듀타운로(동→서)  1.624199  127.054247  37.283538
71   경기도 수원시 영통구 이의동 912-12도         대학로(남→북)  0.421931  127.040784  37.296845
72   경기도 수원시 영통구 이의동 1373-1도       웰빙타운로(서→동)  0.723686  127.038303  37.308580
73  경기도 수원시 장안구 연무동 161-13 도        경수대로(남→북)  0.826423  127.024504  37.290070
74  경기도 수원시 권선구 오목천동 756-1 도        서수원로(남→북)  0.487810  126.983642  37.233788

[75 rows x 5 columns]

* 결측치 확인

In [86]:
dust.isnull().sum()

도로명주소       0
도로명         0
도로재비산심각도    0
경도          0
위도          0
dtype: int64

* 수원시_버스정류장 데이터 전처리

In [87]:
bus = pd.read_csv('1.수원시_버스정류장.csv')
bus.head()

정류장ID     정류장명  정류장유형 환승역타입         위치(동)   쉘터  BIS설치여부  LED  LCD  \
0  201000313     권선구청  시내,마을    일반   경기도 수원시 서둔동  1.0      1.0  1.0  NaN   
1  201000344    중보들공원  시내,마을    일반    경기도 수원시 평동  1.0      NaN  NaN  NaN   
2  201000345  남수원초등학교     시내    일반  경기도 수원시 세류2동  NaN      1.0  NaN  NaN   
3  201000379   신미주아파트     시내    일반   경기도 수원시 금곡동  NaN      1.0  1.0  NaN   
4  201000055   농수산물시장     시내    일반  경기도 수원시 권선1동  NaN      1.0  NaN  1.0   

   LED+LCD복합형  알뜰형  임대형(음성)         lon        lat  
0         NaN  NaN      NaN  126.970983  37.256983  
1         NaN  NaN      NaN  126.991029  37.244605  
2         NaN  1.0      NaN  127.016000  37.251091  
3         NaN  NaN      NaN  126.953845  37.268806  
4         NaN  NaN      NaN  127.029815  37.256755

In [88]:
bus.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1179 entries, 0 to 1178
Data columns (total 14 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   정류장ID       1179 non-null   int64  
 1   정류장명        1179 non-null   object 
 2   정류장유형       1179 non-null   object 
 3   환승역타입       1168 non-null   object 
 4   위치(동)       1179 non-null   object 
 5   쉘터          781 non-null    float64
 6   BIS설치여부     928 non-null    float64
 7   LED         378 non-null    float64
 8   LCD         166 non-null    float64
 9   LED+LCD복합형  358 non-null    float64
 10  알뜰형         26 non-null     float64
 11  임대형(음성)     0 non-null      float64
 12  lon         1179 non-null   float64
 13  lat         1179 non-null   float64
dtypes: float64(9), int64(1), object(4)
memory usage: 129.1+ KB


* 분석 시, BIS 설치가 되어있지 않은 정류소는 제외하기로 함 -> BIS설치여부 1 아닌 행 삭제
* index 리셋

In [89]:
bus.drop(bus[bus['BIS설치여부']!=1].index, inplace=True)
bus.reset_index(inplace=True, drop=True)
bus

정류장ID                  정류장명  정류장유형 환승역타입         위치(동)   쉘터  BIS설치여부  \
0    201000313                  권선구청  시내,마을    일반   경기도 수원시 서둔동  1.0      1.0   
1    201000345               남수원초등학교     시내    일반  경기도 수원시 세류2동  NaN      1.0   
2    201000379                신미주아파트     시내    일반   경기도 수원시 금곡동  NaN      1.0   
3    201000055                농수산물시장     시내    일반  경기도 수원시 권선1동  NaN      1.0   
4    201000268                 곡반중학교     시내    일반   경기도 수원시 곡선동  1.0      1.0   
..         ...                   ...    ...   ...           ...  ...      ...   
923  202000249                 화서오거리     시내    일반  경기도 수원시 화서1동  NaN      1.0   
924  202000241  숙지공원.화서동굿모닝힐.화서다산도서관  시내,마을    일반  경기도 수원시 화서2동  NaN      1.0   
925  202000246           엘지아파트.금강아파트     시내    일반  경기도 수원시 화서2동  1.0      1.0   
926  202000239              꽃뫼신동아아파트  시내,마을    일반  경기도 수원시 화서2동  NaN      1.0   
927  202000238                 진흥아파트  시내,마을    일반  경기도 수원시 화서2동  1.0      1.0   

     LED  LCD  LED+LCD복합형  알뜰형  임대형(음성)         lon        lat  
0    1.0  NaN         NaN  NaN      NaN  126.970983  37.256983  
1    NaN  NaN         NaN  1.0      NaN  127.016000  37.251091  
2    1.0  NaN         NaN  NaN      NaN  126.953845  37.268806  
3    NaN  1.0         NaN  NaN      NaN  127.029815  37.256755  
4    1.0  NaN         NaN  NaN      NaN  127.034254  37.246081  
..   ...  ...         ...  ...      ...         ...        ...  
923  NaN  NaN         1.0  NaN      NaN  126.999697  37.278865  
924  NaN  1.0         NaN  NaN      NaN  126.995823  37.284316  
925  NaN  NaN         1.0  NaN      NaN  126.981714  37.287994  
926  NaN  1.0         NaN  NaN      NaN  126.986076  37.286663  
927  NaN  1.0         NaN  NaN      NaN  126.985096  37.289506  

[928 rows x 14 columns]

In [90]:
bus = bus[['정류장ID','정류장명','lon','lat']]
bus

정류장ID                  정류장명         lon        lat
0    201000313                  권선구청  126.970983  37.256983
1    201000345               남수원초등학교  127.016000  37.251091
2    201000379                신미주아파트  126.953845  37.268806
3    201000055                농수산물시장  127.029815  37.256755
4    201000268                 곡반중학교  127.034254  37.246081
..         ...                   ...         ...        ...
923  202000249                 화서오거리  126.999697  37.278865
924  202000241  숙지공원.화서동굿모닝힐.화서다산도서관  126.995823  37.284316
925  202000246           엘지아파트.금강아파트  126.981714  37.287994
926  202000239              꽃뫼신동아아파트  126.986076  37.286663
927  202000238                 진흥아파트  126.985096  37.289506

[928 rows x 4 columns]

- 버스정류소 좌표 array 로 만들기

In [91]:
bus_array = bus[['정류장ID','lon','lat']]
bus_array['정류장ID'] = bus_array['정류장ID'].astype(str)
bus_array = bus_array[['정류장ID','lon','lat']].to_numpy()
bus_array

/opt/app-root/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


array([['201000313', 126.9709828, 37.25698318],
       ['201000345', 127.0160003, 37.25109066],
       ['201000379', 126.9538449, 37.26880555],
       ...,
       ['202000246', 126.98171380000001, 37.28799446],
       ['202000239', 126.98607630000001, 37.28666265],
       ['202000238', 126.9850962, 37.28950644]], dtype=object)

- 도로 좌표 array로 만들기

In [92]:
road_array = dust[['도로명','경도','위도']]
road_array = road_array.to_numpy()
road_array

array([['경수대로(북→남)', 126.98485500000001, 37.328291899999996],
       ['장다리로(북→남)', 127.0320646, 37.2780249],
       ['장다리로(북→남)', 127.02204640000002, 37.2601145],
       ['덕영대로1190번길(북→남)', 127.0222567, 37.2490729],
       ['덕영대로1246번길(남→북)', 127.02120729999999, 37.243329700000004],
       ['권광로(남→북)', 127.03419950000001, 37.250813],
       ['권광로(남→북)', 127.0347375, 37.2609112],
       ['광교로(남→북)', 127.0381387, 37.2830451],
       ['광교로(남→북)', 127.04524369999999, 37.290933100000004],
       ['창룡대로(동→서)', 127.0594985, 37.298848299999996],
       ['창룡대로(동→서)', 127.0342057, 37.293298],
       ['중부대로(서→동)', 127.01679109999998, 37.273388700000005],
       ['매영로(서→동)', 127.0422636, 37.2694495],
       ['청명북로(남→북)', 127.0754455, 37.251101299999995],
       ['반달로(북→남)', 127.07713000000001, 37.2575773],
       ['봉영로(남→북)', 127.05658259999998, 37.2343334],
       ['광교호수공원로(남→북)', 127.05821920000001, 37.2714136],
       ['도청로(동→서)', 127.05605220000001, 37.2851253],
       ['동탄원천로(북→남)', 127.06278

- 길이확인

In [93]:
print(bus_array.shape[0],road_array.shape[0])

928 75


### Haversine 이용하기

In [94]:
import numbers
import math

class GeoUtil:
    """
    Geographical Utils
    """
    @staticmethod
    def degree2radius(degree):
        return degree * (math.pi/180)
    @staticmethod
    def get_harversion_distance(x1, y1, x2, y2, round_decimal_digits=5):
        """
        경위도 (x1,y1)과 (x2,y2) 점의 거리를 반환
        Harversion Formula 이용하여 2개의 경위도간 거래를 구함(단위:Km)
        """
        if x1 is None or y1 is None or x2 is None or y2 is None:
            return None
        assert isinstance(x1, numbers.Number) and -180 <= x1 and x1 <= 180
        assert isinstance(y1, numbers.Number) and  -90 <= y1 and y1 <=  90
        assert isinstance(x2, numbers.Number) and -180 <= x2 and x2 <= 180
        assert isinstance(y2, numbers.Number) and  -90 <= y2 and y2 <=  90

        R = 6371 # 지구의 반경(단위: km)
        dLon = GeoUtil.degree2radius(x2-x1)    
        dLat = GeoUtil.degree2radius(y2-y1)

        a = math.sin(dLat/2) * math.sin(dLat/2) \
            + (math.cos(GeoUtil.degree2radius(y1)) \
              *math.cos(GeoUtil.degree2radius(y2)) \
              *math.sin(dLon/2) * math.sin(dLon/2))
        b = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
        return round(R * b, round_decimal_digits)

In [95]:
index_1km = [] 
for i in range(bus_array.shape[0]):
    each_road=[]
    for j in range(road_array.shape[0]):
        h_dis = GeoUtil.get_harversion_distance(bus_array[i,1], bus_array[i,2], road_array[j,1],road_array[j,2])
        if h_dis<=1:  
            each_road.append(road_array[j,0])
            
    index_1km.append(each_road)

In [96]:
index_1km[:10]

[['호매실로(동→서)'],
 ['덕영대로1190번길(북→남)', '덕영대로1246번길(남→북)'],
 ['칠보로(동→서)', '매실로(동→서)', '매곡로(서→동)', '금호로(서→동)', '권선로(서→동)'],
 ['장다리로(북→남)', '권광로(남→북)', '권광로(남→북)', '권중로(북→남)', '권선로(북→남)'],
 ['권광로(남→북)', '덕영대로(동→서)'],
 ['금곡로(서→동)', '매곡로(서→동)', '금호로(서→동)'],
 ['상률로(서→동)', '천천로(북→남)'],
 ['정자천로(서→동)', '화산로(남→북)', '덕영대로(동→서)', '수성로(동→서)', '수성로(동→서)'],
 ['칠보로(동→서)', '매실로(동→서)', '매곡로(서→동)', '금호로(서→동)', '권선로(서→동)'],
 ['칠보로(동→서)', '매실로(동→서)', '매곡로(서→동)', '금호로(서→동)', '권선로(서→동)']]

- bus 데이터에 위에서 구한 도로 리스트를 컬럼으로 생성

In [97]:
bus['반경1km이내_도로'] = index_1km
bus

/opt/app-root/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


정류장ID                  정류장명         lon        lat  \
0    201000313                  권선구청  126.970983  37.256983   
1    201000345               남수원초등학교  127.016000  37.251091   
2    201000379                신미주아파트  126.953845  37.268806   
3    201000055                농수산물시장  127.029815  37.256755   
4    201000268                 곡반중학교  127.034254  37.246081   
..         ...                   ...         ...        ...   
923  202000249                 화서오거리  126.999697  37.278865   
924  202000241  숙지공원.화서동굿모닝힐.화서다산도서관  126.995823  37.284316   
925  202000246           엘지아파트.금강아파트  126.981714  37.287994   
926  202000239              꽃뫼신동아아파트  126.986076  37.286663   
927  202000238                 진흥아파트  126.985096  37.289506   

                                            반경1km이내_도로  
0                                          [호매실로(동→서)]  
1                   [덕영대로1190번길(북→남), 덕영대로1246번길(남→북)]  
2    [칠보로(동→서), 매실로(동→서), 매곡로(서→동), 금호로(서→동), 권선로(서...  
3    [장다리로(북→남), 권광로(남→북), 권광로(남→북), 권중로(북→남), 권선로(...  
4                                [권광로(남→북), 덕영대로(동→서)]  
..                                                 ...  
923                                                 []  
924                               [화산로(남→북), 수성로(동→서)]  
925  [정자천로(서→동), 화산로(남→북), 덕영대로(동→서), 수성로(동→서), 수성로...  
926  [정자천로(서→동), 화산로(남→북), 덕영대로(동→서), 수성로(동→서), 수성로...  
927  [정자천로(서→동), 화산로(남→북), 덕영대로(동→서), 만석로(서→동), 수성로...  

[928 rows x 5 columns]

### 리스트로 구성된 도로 목록을 행으로 분리하기

In [98]:
for i in range(len(bus)):
    bus['반경1km이내_도로'][i] = ' '.join(bus['반경1km이내_도로'][i])
bus

/opt/app-root/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/app-root/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3343: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


정류장ID                  정류장명         lon        lat  \
0    201000313                  권선구청  126.970983  37.256983   
1    201000345               남수원초등학교  127.016000  37.251091   
2    201000379                신미주아파트  126.953845  37.268806   
3    201000055                농수산물시장  127.029815  37.256755   
4    201000268                 곡반중학교  127.034254  37.246081   
..         ...                   ...         ...        ...   
923  202000249                 화서오거리  126.999697  37.278865   
924  202000241  숙지공원.화서동굿모닝힐.화서다산도서관  126.995823  37.284316   
925  202000246           엘지아파트.금강아파트  126.981714  37.287994   
926  202000239              꽃뫼신동아아파트  126.986076  37.286663   
927  202000238                 진흥아파트  126.985096  37.289506   

                                            반경1km이내_도로  
0                                            호매실로(동→서)  
1                      덕영대로1190번길(북→남) 덕영대로1246번길(남→북)  
2         칠보로(동→서) 매실로(동→서) 매곡로(서→동) 금호로(서→동) 권선로(서→동)  
3        장다리로(북→남) 권광로(남→북) 권광로(남→북) 권중로(북→남) 권선로(북→남)  
4                                   권광로(남→북) 덕영대로(동→서)  
..                                                 ...  
923                                                     
924                                  화산로(남→북) 수성로(동→서)  
925     정자천로(서→동) 화산로(남→북) 덕영대로(동→서) 수성로(동→서) 수성로(동→서)  
926     정자천로(서→동) 화산로(남→북) 덕영대로(동→서) 수성로(동→서) 수성로(동→서)  
927  정자천로(서→동) 화산로(남→북) 덕영대로(동→서) 만석로(서→동) 수성로(동→서)...  

[928 rows x 5 columns]

In [99]:
split =bus.반경1km이내_도로.str.split(' ')
split

0                                            [호매실로(동→서)]
1                     [덕영대로1190번길(북→남), 덕영대로1246번길(남→북)]
2      [칠보로(동→서), 매실로(동→서), 매곡로(서→동), 금호로(서→동), 권선로(서...
3      [장다리로(북→남), 권광로(남→북), 권광로(남→북), 권중로(북→남), 권선로(...
4                                  [권광로(남→북), 덕영대로(동→서)]
                             ...                        
923                                                   []
924                                 [화산로(남→북), 수성로(동→서)]
925    [정자천로(서→동), 화산로(남→북), 덕영대로(동→서), 수성로(동→서), 수성로...
926    [정자천로(서→동), 화산로(남→북), 덕영대로(동→서), 수성로(동→서), 수성로...
927    [정자천로(서→동), 화산로(남→북), 덕영대로(동→서), 만석로(서→동), 수성로...
Name: 반경1km이내_도로, Length: 928, dtype: object

- splite을 Series로 만들기

In [100]:
split = split.apply(lambda x: pd.Series(x))
split

0                1          2         3         4  \
0          호매실로(동→서)              NaN        NaN       NaN       NaN   
1    덕영대로1190번길(북→남)  덕영대로1246번길(남→북)        NaN       NaN       NaN   
2           칠보로(동→서)         매실로(동→서)   매곡로(서→동)  금호로(서→동)  권선로(서→동)   
3          장다리로(북→남)         권광로(남→북)   권광로(남→북)  권중로(북→남)  권선로(북→남)   
4           권광로(남→북)        덕영대로(동→서)        NaN       NaN       NaN   
..               ...              ...        ...       ...       ...   
923                               NaN        NaN       NaN       NaN   
924         화산로(남→북)         수성로(동→서)        NaN       NaN       NaN   
925        정자천로(서→동)         화산로(남→북)  덕영대로(동→서)  수성로(동→서)  수성로(동→서)   
926        정자천로(서→동)         화산로(남→북)  덕영대로(동→서)  수성로(동→서)  수성로(동→서)   
927        정자천로(서→동)         화산로(남→북)  덕영대로(동→서)  만석로(서→동)  수성로(동→서)   

            5    6  
0         NaN  NaN  
1         NaN  NaN  
2         NaN  NaN  
3         NaN  NaN  
4         NaN  NaN  
..        ...  ...  
923       NaN  NaN  
924       NaN  NaN  
925       NaN  NaN  
926       NaN  NaN  
927  수성로(동→서)  NaN  

[928 rows x 7 columns]

In [101]:
split.stack().reset_index(level=1, drop=True)

0            호매실로(동→서)
1      덕영대로1190번길(북→남)
1      덕영대로1246번길(남→북)
2             칠보로(동→서)
2             매실로(동→서)
            ...       
927           화산로(남→북)
927          덕영대로(동→서)
927           만석로(서→동)
927           수성로(동→서)
927           수성로(동→서)
Length: 2497, dtype: object

In [102]:
split = split.stack().reset_index(level=1, drop=True).to_frame('도로명')
split

도로명
0          호매실로(동→서)
1    덕영대로1190번길(북→남)
1    덕영대로1246번길(남→북)
2           칠보로(동→서)
2           매실로(동→서)
..               ...
927         화산로(남→북)
927        덕영대로(동→서)
927         만석로(서→동)
927         수성로(동→서)
927         수성로(동→서)

[2497 rows x 1 columns]

In [103]:
bus = bus.merge(split, left_index=True, right_index=True, how='left')
bus.head()

정류장ID     정류장명         lon        lat  \
0  201000313     권선구청  126.970983  37.256983   
1  201000345  남수원초등학교  127.016000  37.251091   
1  201000345  남수원초등학교  127.016000  37.251091   
2  201000379   신미주아파트  126.953845  37.268806   
2  201000379   신미주아파트  126.953845  37.268806   

                                     반경1km이내_도로              도로명  
0                                     호매실로(동→서)        호매실로(동→서)  
1               덕영대로1190번길(북→남) 덕영대로1246번길(남→북)  덕영대로1190번길(북→남)  
1               덕영대로1190번길(북→남) 덕영대로1246번길(남→북)  덕영대로1246번길(남→북)  
2  칠보로(동→서) 매실로(동→서) 매곡로(서→동) 금호로(서→동) 권선로(서→동)         칠보로(동→서)  
2  칠보로(동→서) 매실로(동→서) 매곡로(서→동) 금호로(서→동) 권선로(서→동)         매실로(동→서)

- 전처리 전 컬럼 삭제

In [104]:
del bus['반경1km이내_도로']

- 버스정류소와 도로재비산먼지 merge

In [105]:
bus_with_dust = pd.merge(bus,dust,how='left',on=('도로명'))
bus_with_dust

정류장ID     정류장명         lon        lat              도로명  \
0     201000313     권선구청  126.970983  37.256983        호매실로(동→서)   
1     201000345  남수원초등학교  127.016000  37.251091  덕영대로1190번길(북→남)   
2     201000345  남수원초등학교  127.016000  37.251091  덕영대로1246번길(남→북)   
3     201000379   신미주아파트  126.953845  37.268806         칠보로(동→서)   
4     201000379   신미주아파트  126.953845  37.268806         매실로(동→서)   
...         ...      ...         ...        ...              ...   
3385  202000238    진흥아파트  126.985096  37.289506         수성로(동→서)   
3386  202000238    진흥아파트  126.985096  37.289506         수성로(동→서)   
3387  202000238    진흥아파트  126.985096  37.289506         수성로(동→서)   
3388  202000238    진흥아파트  126.985096  37.289506         수성로(동→서)   
3389  202000238    진흥아파트  126.985096  37.289506         수성로(동→서)   

                         도로명주소  도로재비산심각도          경도         위도  
0     경기도 수원시 권선구 고색동 893-74 도  0.633244  126.973769  37.256160  
1        경기도 수원시 권선구 권선동 1212도  0.840071  127.022257  37.249073  
2     경기도 수원시 권선구 권선동 1337-9 도  1.556415  127.021207  37.243330  
3     경기도 수원시 권선구 호매실동 1323-3도  0.355506  126.959238  37.270403  
4     경기도 수원시 권선구 호매실동 36-11 답  3.504247  126.961310  37.269813  
...                        ...       ...         ...        ...  
3385    경기도 수원시 팔달구 화서동 360-1도  9.001188  126.992250  37.286912  
3386    경기도 수원시 권선구 서둔동 279-1도  1.838009  126.982032  37.283717  
3387   경기도 수원시 장안구 조원동 533-11도  0.861186  127.020540  37.296622  
3388    경기도 수원시 팔달구 화서동 360-1도  9.001188  126.992250  37.286912  
3389    경기도 수원시 권선구 서둔동 279-1도  1.838009  126.982032  37.283717  

[3390 rows x 9 columns]

- 그룹화해서 평균을 구할 변수만 선택
- 이떄, 'lon', 'lat'은 버스정류장의 위도,경도 / '경도', '위도'는 도로의 경도,위도

In [106]:
bus_group = bus_with_dust[['정류장ID','도로재비산심각도']]
bus_group.head(10)

정류장ID  도로재비산심각도
0  201000313  0.633244
1  201000345  0.840071
2  201000345  1.556415
3  201000379  0.355506
4  201000379  3.504247
5  201000379  2.764698
6  201000379  0.767705
7  201000379  0.560147
8  201000055  2.863450
9  201000055  1.862622

- 정류장id로 그룹화

In [107]:
bus_group=bus_group.groupby(['정류장ID'], as_index=False).mean()
bus_group

정류장ID  도로재비산심각도
0    200000005  0.553726
1    200000006  2.553933
2    200000007  0.497072
3    200000008  0.723686
4    200000009       NaN
..         ...       ...
923  203000424  0.528371
924  203000438  0.800966
925  203000439  0.800966
926  203000449  0.534804
927  203000450  0.678509

[928 rows x 2 columns]

- 평균을 구하지 않은 변수들 중 중복값 삭제

In [108]:
unique_id = bus_with_dust[['정류장ID','정류장명','lon','lat','도로명','도로명주소','경도','위도']].drop_duplicates() 
unique_id

정류장ID     정류장명         lon        lat              도로명  \
0     201000313     권선구청  126.970983  37.256983        호매실로(동→서)   
1     201000345  남수원초등학교  127.016000  37.251091  덕영대로1190번길(북→남)   
2     201000345  남수원초등학교  127.016000  37.251091  덕영대로1246번길(남→북)   
3     201000379   신미주아파트  126.953845  37.268806         칠보로(동→서)   
4     201000379   신미주아파트  126.953845  37.268806         매실로(동→서)   
...         ...      ...         ...        ...              ...   
3382  202000238    진흥아파트  126.985096  37.289506        덕영대로(동→서)   
3383  202000238    진흥아파트  126.985096  37.289506         만석로(서→동)   
3384  202000238    진흥아파트  126.985096  37.289506         수성로(동→서)   
3385  202000238    진흥아파트  126.985096  37.289506         수성로(동→서)   
3386  202000238    진흥아파트  126.985096  37.289506         수성로(동→서)   

                         도로명주소          경도         위도  
0     경기도 수원시 권선구 고색동 893-74 도  126.973769  37.256160  
1        경기도 수원시 권선구 권선동 1212도  127.022257  37.249073  
2     경기도 수원시 권선구 권선동 1337-9 도  127.021207  37.243330  
3     경기도 수원시 권선구 호매실동 1323-3도  126.959238  37.270403  
4     경기도 수원시 권선구 호매실동 36-11 답  126.961310  37.269813  
...                        ...         ...        ...  
3382    경기도 수원시 장안구 정자동 931-2천  126.988541  37.292794  
3383      경기도 수원시 장안구 천천동 535도  126.979508  37.297182  
3384   경기도 수원시 장안구 조원동 533-11도  127.020540  37.296622  
3385    경기도 수원시 팔달구 화서동 360-1도  126.992250  37.286912  
3386    경기도 수원시 권선구 서둔동 279-1도  126.982032  37.283717  

[3252 rows x 8 columns]

- sum 결과와 '정류장ID','정류장명','시군구명','읍면동명','link_id' merge

In [109]:
busDust_mean = pd.merge(bus_group,unique_id, how='left',on='정류장ID')
busDust_mean

정류장ID  도로재비산심각도                  정류장명         lon        lat  \
0     200000005  0.553726        광교산입구.경기대수원캠퍼스  127.031546  37.301462   
1     200000005  0.553726        광교산입구.경기대수원캠퍼스  127.031546  37.301462   
2     200000005  0.553726        광교산입구.경기대수원캠퍼스  127.031546  37.301462   
3     200000005  0.553726        광교산입구.경기대수원캠퍼스  127.031546  37.301462   
4     200000006  2.553933  광교공원.경기대수원캠퍼스입구.연무시장  127.029464  37.300014   
...         ...       ...                   ...         ...        ...   
3247  203000439  0.800966              광교카페거리입구  127.056369  37.295078   
3248  203000439  0.800966              광교카페거리입구  127.056369  37.295078   
3249  203000439  0.800966              광교카페거리입구  127.056369  37.295078   
3250  203000449  0.534804      대선초.e편한세상영통2차아파트  127.065462  37.241033   
3251  203000450  0.678509             살구골.서광아파트  127.073142  37.247341   

              도로명                     도로명주소          경도         위도  
0       창룡대로(동→서)     경기도 수원시 영통구 이의동 1374도  127.059499  37.298848  
1       창룡대로(동→서)      경기도 수원시 팔달구 우만동 459도  127.034206  37.293298  
2        대학로(남→북)   경기도 수원시 영통구 이의동 912-12도  127.040784  37.296845  
3      웰빙타운로(서→동)   경기도 수원시 영통구 이의동 1373-1도  127.038303  37.308580  
4       창룡대로(동→서)     경기도 수원시 영통구 이의동 1374도  127.059499  37.298848  
...           ...                       ...         ...        ...  
3247    창룡대로(동→서)      경기도 수원시 팔달구 우만동 459도  127.034206  37.293298  
3248  센트럴타운로(동→서)  경기도 수원시 영통구 이의동 1381-23도  127.059786  37.288306  
3249  센트럴파크로(북→남)   경기도 수원시 영통구 이의동 1379-5도  127.053136  37.295280  
3250     영통로(남→북)     경기도 수원시 영통구 망포동 678 구  127.059246  37.235083  
3251    청명북로(남→북)     경기도 수원시 영통구 영통동 1079도  127.075446  37.251101  

[3252 rows x 9 columns]

In [110]:
busDust_mean_csv = busDust_mean[['정류장ID','도로재비산심각도']]
busDust_mean_csv=busDust_mean_csv.drop_duplicates() 
busDust_mean_csv.to_csv('data/busDust_mean.txt',sep=',',index=False)
busDust_mean_csv.reset_index(drop=True,inplace=True)

### 시각화

- 시각화를 위해 결측값 제거

In [111]:
busDust_mean_notnull = busDust_mean.dropna()
busDust_mean_notnull.reset_index(inplace=True,drop=True)
busDust_mean_notnull

정류장ID  도로재비산심각도                  정류장명         lon        lat  \
0     200000005  0.553726        광교산입구.경기대수원캠퍼스  127.031546  37.301462   
1     200000005  0.553726        광교산입구.경기대수원캠퍼스  127.031546  37.301462   
2     200000005  0.553726        광교산입구.경기대수원캠퍼스  127.031546  37.301462   
3     200000005  0.553726        광교산입구.경기대수원캠퍼스  127.031546  37.301462   
4     200000006  2.553933  광교공원.경기대수원캠퍼스입구.연무시장  127.029464  37.300014   
...         ...       ...                   ...         ...        ...   
3176  203000439  0.800966              광교카페거리입구  127.056369  37.295078   
3177  203000439  0.800966              광교카페거리입구  127.056369  37.295078   
3178  203000439  0.800966              광교카페거리입구  127.056369  37.295078   
3179  203000449  0.534804      대선초.e편한세상영통2차아파트  127.065462  37.241033   
3180  203000450  0.678509             살구골.서광아파트  127.073142  37.247341   

              도로명                     도로명주소          경도         위도  
0       창룡대로(동→서)     경기도 수원시 영통구 이의동 1374도  127.059499  37.298848  
1       창룡대로(동→서)      경기도 수원시 팔달구 우만동 459도  127.034206  37.293298  
2        대학로(남→북)   경기도 수원시 영통구 이의동 912-12도  127.040784  37.296845  
3      웰빙타운로(서→동)   경기도 수원시 영통구 이의동 1373-1도  127.038303  37.308580  
4       창룡대로(동→서)     경기도 수원시 영통구 이의동 1374도  127.059499  37.298848  
...           ...                       ...         ...        ...  
3176    창룡대로(동→서)      경기도 수원시 팔달구 우만동 459도  127.034206  37.293298  
3177  센트럴타운로(동→서)  경기도 수원시 영통구 이의동 1381-23도  127.059786  37.288306  
3178  센트럴파크로(북→남)   경기도 수원시 영통구 이의동 1379-5도  127.053136  37.295280  
3179     영통로(남→북)     경기도 수원시 영통구 망포동 678 구  127.059246  37.235083  
3180    청명북로(남→북)     경기도 수원시 영통구 영통동 1079도  127.075446  37.251101  

[3181 rows x 9 columns]

- 시각화에서 구를 구분하기 위해 데이터 불러오기

In [112]:
df29 = gpd.read_file("29.수원시_법정경계(시군구).geojson") 
df29.head()

SIG_CD SIG_KOR_NM                                           geometry
0  41111    수원시 장안구  MULTIPOLYGON (((127.01976 37.35104, 127.02008 ...
1  41113    수원시 권선구  MULTIPOLYGON (((127.03723 37.23684, 127.03679 ...
2  41115    수원시 팔달구  MULTIPOLYGON (((127.03582 37.29361, 127.03644 ...
3  41117    수원시 영통구  MULTIPOLYGON (((127.07484 37.26588, 127.07488 ...

In [113]:
center = [37.26404308610561, 127.02347288584993]
m = folium.Map(location=center, zoom_start=13,tiles='cartodbpositron')
style1 = {'fillColor': False, 'color': '#AAAAAA'}
style2 = {'fillColor': '#AAAAAA', 'color': '#AAAAAA'}


folium.GeoJson(data=df29["geometry"], style_function=lambda x:style1).add_to(m) #수원시 구별 경계


for n in busDust_mean_notnull.index:
    folium.CircleMarker([busDust_mean_notnull['위도'][n], busDust_mean_notnull['경도'][n]], 
                        radius = busDust_mean_notnull['도로재비산심각도'][n]*4, 
                        color=busDust_mean_notnull['도로재비산심각도'][n]*2, fill_color='red', fill=True).add_to(m)
    
for n in busDust_mean_notnull.index:
    folium.CircleMarker([busDust_mean_notnull['lat'][n], busDust_mean_notnull['lon'][n]], 
                        radius = 1, 
                        color='#444444', fill_color='#444444', fill=True).add_to(m)

* 회색 circle -> 버스정류장 좌표
* 빨간색 circle marker -> 도로재비산먼지가 측정된 좌표 (크기가 클수록 도로재비산먼지 심각도가 높음)

### 시각화 결과:  팔달구와 인근 도로의 도로재비산먼지 심각도가 높음

###  2. 추정 교통량 데이터와 정류장 연결

- 추정 교텅량 데이터와 상세도로망 데이터 merge

In [114]:
traffic = pd.read_csv('23.수원시_평일_일별_시간대별_추정교통량_LV6.csv')
traffic

/opt/app-root/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


상세도로망_LinkID  도로등급   링크길이       도로명  시도명    시군구명 읍면동명 시간적범위  전체_추정교통량  \
0        47834670601   103  0.004     매송고색로  경기도  수원시권선구   평동     0       128   
1        47834670602   103  0.004     매송고색로  경기도  수원시권선구   평동     0       114   
2        47834131701   103  0.007     매송고색로  경기도  수원시권선구   평동     0       122   
3        47834131702   103  0.007     매송고색로  경기도  수원시권선구   평동     0       131   
4        47834086501   103  0.012     매송고색로  경기도  수원시권선구   평동     0       117   
...              ...   ...    ...       ...  ...     ...  ...   ...       ...   
291695   47846651301   108  0.206  수원광명고속도로  경기도  수원시권선구  입북동    23       238   
291696   47846651401   108  0.246  수원광명고속도로  경기도  수원시권선구  입북동    23       268   
291697   47846662301   108  0.505  수원광명고속도로  경기도  수원시권선구  입북동    23       112   
291698   47846662101   108  0.610  수원광명고속도로  경기도  수원시권선구  입북동    23       136   
291699   47846651201   108  0.968  수원광명고속도로  경기도  수원시권선구  입북동    23       240   

        승용차_추정교통량  버스_추정교통량  화물차_추정교통량  
0             109         1         19  
1             102         1         11  
2             109         1         12  
3             111         1         19  
4             105         0         11  
...           ...       ...        ...  
291695        159         1         79  
291696        221         1         46  
291697         84         0         28  
291698         93         1         42  
291699        168         1         72  

[291700 rows x 12 columns]

시간적 범위 fulltime만 남기고 인덱스 리셋하기

In [115]:
traffic = traffic[traffic['시간적범위']=='fulltime']
traffic.reset_index(inplace=True,drop=True)
traffic

상세도로망_LinkID  도로등급   링크길이       도로명  시도명    시군구명 읍면동명     시간적범위  \
0       47835381601   101  3.130  수원광명고속도로  경기도  수원시권선구  금곡동  fulltime   
1       47835381901   101  3.130  수원광명고속도로  경기도  수원시권선구  금곡동  fulltime   
2       47846654001   101  0.015  수원광명고속도로  경기도  수원시권선구  입북동  fulltime   
3       47846654101   101  0.036  수원광명고속도로  경기도  수원시권선구  입북동  fulltime   
4       47846654301   101  0.036  수원광명고속도로  경기도  수원시권선구  입북동  fulltime   
...             ...   ...    ...       ...  ...     ...  ...       ...   
11663   47844249301   108  0.456    영동고속도로  경기도  수원시장안구  파장동  fulltime   
11664   47844249801   108  0.648    영동고속도로  경기도  수원시장안구  파장동  fulltime   
11665   47844249701   108  0.659    영동고속도로  경기도  수원시장안구  파장동  fulltime   
11666   47844226401   108  0.664    영동고속도로  경기도  수원시장안구  파장동  fulltime   
11667   47844249901   108  1.007    영동고속도로  경기도  수원시장안구  파장동  fulltime   

       전체_추정교통량  승용차_추정교통량  버스_추정교통량  화물차_추정교통량  
0         11690       8588       424       2678  
1         14564      10664       696       3203  
2         17031      12607       604       3820  
3         18025      13367       612       4046  
4         19989      14868       762       4359  
...         ...        ...       ...        ...  
11663     16417      12142       446       3830  
11664      3746       2831       104        812  
11665     19727      14766       907       4054  
11666     26583      19942      1047       5594  
11667     19910      14936       784       4190  

[11668 rows x 12 columns]

상세도로망_LinkID feature를 str형식으로 전환

In [116]:
traffic['상세도로망_LinkID'] = traffic['상세도로망_LinkID'].astype(str)

/opt/app-root/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


상세도로망_linkID와 상세도로망 데이터의 link_id를 기준으로 merge하기 위해 상세도로망_linkID 일부 변경

In [117]:
traffic['link_id']=''
for i in range(len(traffic)):
    traffic['link_id'][i] = traffic['상세도로망_LinkID'][i][:-2]
traffic

/opt/app-root/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/app-root/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/app-root/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3343: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http

상세도로망_LinkID  도로등급   링크길이       도로명  시도명    시군구명 읍면동명     시간적범위  \
0      47835381601   101  3.130  수원광명고속도로  경기도  수원시권선구  금곡동  fulltime   
1      47835381901   101  3.130  수원광명고속도로  경기도  수원시권선구  금곡동  fulltime   
2      47846654001   101  0.015  수원광명고속도로  경기도  수원시권선구  입북동  fulltime   
3      47846654101   101  0.036  수원광명고속도로  경기도  수원시권선구  입북동  fulltime   
4      47846654301   101  0.036  수원광명고속도로  경기도  수원시권선구  입북동  fulltime   
...            ...   ...    ...       ...  ...     ...  ...       ...   
11663  47844249301   108  0.456    영동고속도로  경기도  수원시장안구  파장동  fulltime   
11664  47844249801   108  0.648    영동고속도로  경기도  수원시장안구  파장동  fulltime   
11665  47844249701   108  0.659    영동고속도로  경기도  수원시장안구  파장동  fulltime   
11666  47844226401   108  0.664    영동고속도로  경기도  수원시장안구  파장동  fulltime   
11667  47844249901   108  1.007    영동고속도로  경기도  수원시장안구  파장동  fulltime   

       전체_추정교통량  승용차_추정교통량  버스_추정교통량  화물차_추정교통량    link_id  
0         11690       8588       424       2678  478353816  
1         14564      10664       696       3203  478353819  
2         17031      12607       604       3820  478466540  
3         18025      13367       612       4046  478466541  
4         19989      14868       762       4359  478466543  
...         ...        ...       ...        ...        ...  
11663     16417      12142       446       3830  478442493  
11664      3746       2831       104        812  478442498  
11665     19727      14766       907       4054  478442497  
11666     26583      19942      1047       5594  478442264  
11667     19910      14936       784       4190  478442499  

[11668 rows x 13 columns]

분석에 필요한 컬럼만 선택

In [118]:
traffic = traffic[['link_id','시군구명','읍면동명','전체_추정교통량','승용차_추정교통량','버스_추정교통량','화물차_추정교통량','상세도로망_LinkID']]
traffic

link_id    시군구명 읍면동명  전체_추정교통량  승용차_추정교통량  버스_추정교통량  화물차_추정교통량  \
0      478353816  수원시권선구  금곡동     11690       8588       424       2678   
1      478353819  수원시권선구  금곡동     14564      10664       696       3203   
2      478466540  수원시권선구  입북동     17031      12607       604       3820   
3      478466541  수원시권선구  입북동     18025      13367       612       4046   
4      478466543  수원시권선구  입북동     19989      14868       762       4359   
...          ...     ...  ...       ...        ...       ...        ...   
11663  478442493  수원시장안구  파장동     16417      12142       446       3830   
11664  478442498  수원시장안구  파장동      3746       2831       104        812   
11665  478442497  수원시장안구  파장동     19727      14766       907       4054   
11666  478442264  수원시장안구  파장동     26583      19942      1047       5594   
11667  478442499  수원시장안구  파장동     19910      14936       784       4190   

      상세도로망_LinkID  
0      47835381601  
1      47835381901  
2      47846654001  
3      47846654101  
4      47846654301  
...            ...  
11663  47844249301  
11664  47844249801  
11665  47844249701  
11666  47844226401  
11667  47844249901  

[11668 rows x 8 columns]

상세도로망 전처리

In [119]:
linkid = gpd.read_file('22.수원시_상세도로망_LV6.geojson')
linkid

link_id max_speed road_name road_no road_rank link_type road_type  \
0     478300330        60     삼천병마로       0       107     32768         0   
1     478300581        80     매송고색로      43       103     32768         3   
2     478310046        80     매송고색로      43       103     32768         0   
3     478310049        80     매송고색로      43       103     32768         0   
4     478310050        80     매송고색로      43       103     32768         0   
...         ...       ...       ...     ...       ...       ...       ...   
6648  571492408        60     동탄원천로       0       107         1         0   
6649  571492534       100    영동고속도로      50       101         1         0   
6650  571492535       100    영동고속도로      50       101         1         3   
6651  571492538       100    영동고속도로      50       101         1         0   
6652  571492539       100    영동고속도로      50       101         1         3   

     facil_name up_lanes dw_lanes oneway length width car_lane num_cross  \
0          None        1        1      0  0.081     2        0         0   
1          오목천교        3        3      0  0.056     4        0         0   
2          None        3        3      0  0.043     4        0         0   
3          None        3        3      0  0.035     4        0         0   
4          None        3        3      0  0.035     4        0         1   
...         ...      ...      ...    ...    ...   ...      ...       ...   
6648       None        2        0      1  0.235     2        0         0   
6649       None        4        0      1  0.205     3        1         0   
6650       None        4        0      1  0.224     3        1         0   
6651       None        5        0      1  0.154     4        1         0   
6652       None        5        0      1  0.225     4        1         0   

     barrier   up_its_id   dw_its_id  \
0          0  2010013500  2010013600   
1          3  2010001400  2010001300   
2         15  2010001703  2010001702   
3         15  2010001702  2010001703   
4         15  2010001704  2010001705   
...      ...         ...         ...   
6648       0  2030062800           0   
6649       0  2030005401           0   
6650       0  2030005401           0   
6651       0  2030005301           0   
6652       0  2030005301           0   

                                               geometry  
0     MULTILINESTRING ((126.96360 37.24516, 126.9633...  
1     MULTILINESTRING ((126.96904 37.24752, 126.9696...  
2     MULTILINESTRING ((126.98178 37.25112, 126.9822...  
3     MULTILINESTRING ((126.98263 37.25136, 126.9822...  
4     MULTILINESTRING ((126.98072 37.25081, 126.9805...  
...                                                 ...  
6648  MULTILINESTRING ((127.06172 37.27031, 127.0618...  
6649  MULTILINESTRING ((127.02405 37.31358, 127.0237...  
6650  MULTILINESTRING ((127.02183 37.31304, 127.0215...  
6651  MULTILINESTRING ((127.02189 37.31288, 127.0225...  
6652  MULTILINESTRING ((127.01950 37.31220, 127.0216...  

[6653 rows x 19 columns]

분석에 필요한 컬럼만 선택

In [120]:
linkid = linkid[['link_id','geometry']]
linkid

link_id                                           geometry
0     478300330  MULTILINESTRING ((126.96360 37.24516, 126.9633...
1     478300581  MULTILINESTRING ((126.96904 37.24752, 126.9696...
2     478310046  MULTILINESTRING ((126.98178 37.25112, 126.9822...
3     478310049  MULTILINESTRING ((126.98263 37.25136, 126.9822...
4     478310050  MULTILINESTRING ((126.98072 37.25081, 126.9805...
...         ...                                                ...
6648  571492408  MULTILINESTRING ((127.06172 37.27031, 127.0618...
6649  571492534  MULTILINESTRING ((127.02405 37.31358, 127.0237...
6650  571492535  MULTILINESTRING ((127.02183 37.31304, 127.0215...
6651  571492538  MULTILINESTRING ((127.02189 37.31288, 127.0225...
6652  571492539  MULTILINESTRING ((127.01950 37.31220, 127.0216...

[6653 rows x 2 columns]

link id를 기준으로 inner_join 실행

In [121]:
traffic_link = pd.merge(traffic,linkid,how='left',on=('link_id'))
traffic_link

link_id    시군구명 읍면동명  전체_추정교통량  승용차_추정교통량  버스_추정교통량  화물차_추정교통량  \
0      478353816  수원시권선구  금곡동     11690       8588       424       2678   
1      478353819  수원시권선구  금곡동     14564      10664       696       3203   
2      478466540  수원시권선구  입북동     17031      12607       604       3820   
3      478466541  수원시권선구  입북동     18025      13367       612       4046   
4      478466543  수원시권선구  입북동     19989      14868       762       4359   
...          ...     ...  ...       ...        ...       ...        ...   
11663  478442493  수원시장안구  파장동     16417      12142       446       3830   
11664  478442498  수원시장안구  파장동      3746       2831       104        812   
11665  478442497  수원시장안구  파장동     19727      14766       907       4054   
11666  478442264  수원시장안구  파장동     26583      19942      1047       5594   
11667  478442499  수원시장안구  파장동     19910      14936       784       4190   

      상세도로망_LinkID                                           geometry  
0      47835381601  MULTILINESTRING ((126.94994 37.28007, 126.9499...  
1      47835381901  MULTILINESTRING ((126.94972 37.25199, 126.9496...  
2      47846654001  MULTILINESTRING ((126.94859 37.28954, 126.9486...  
3      47846654101  MULTILINESTRING ((126.94851 37.28987, 126.9485...  
4      47846654301  MULTILINESTRING ((126.94876 37.28956, 126.9486...  
...            ...                                                ...  
11663  47844249301  MULTILINESTRING ((126.99392 37.31739, 126.9936...  
11664  47844249801  MULTILINESTRING ((126.99114 37.31845, 126.9915...  
11665  47844249701  MULTILINESTRING ((126.99133 37.32058, 126.9912...  
11666  47844226401  MULTILINESTRING ((126.98800 37.32578, 126.9882...  
11667  47844249901  MULTILINESTRING ((126.99125 37.32040, 126.9912...  

[11668 rows x 9 columns]

결측치 학인

In [122]:
traffic_link.isnull().sum()

link_id         0
시군구명            0
읍면동명            0
전체_추정교통량        0
승용차_추정교통량       0
버스_추정교통량        0
화물차_추정교통량       0
상세도로망_LinkID    0
geometry        0
dtype: int64

merge 완료된 데이터 확인

In [123]:
traffic_link.head()

link_id    시군구명 읍면동명  전체_추정교통량  승용차_추정교통량  버스_추정교통량  화물차_추정교통량  \
0  478353816  수원시권선구  금곡동     11690       8588       424       2678   
1  478353819  수원시권선구  금곡동     14564      10664       696       3203   
2  478466540  수원시권선구  입북동     17031      12607       604       3820   
3  478466541  수원시권선구  입북동     18025      13367       612       4046   
4  478466543  수원시권선구  입북동     19989      14868       762       4359   

  상세도로망_LinkID                                           geometry  
0  47835381601  MULTILINESTRING ((126.94994 37.28007, 126.9499...  
1  47835381901  MULTILINESTRING ((126.94972 37.25199, 126.9496...  
2  47846654001  MULTILINESTRING ((126.94859 37.28954, 126.9486...  
3  47846654101  MULTILINESTRING ((126.94851 37.28987, 126.9485...  
4  47846654301  MULTILINESTRING ((126.94876 37.28956, 126.9486...

버스정류소 데이터와 결합하기 위해, geometry의 두 좌표의 가운데에 위치한 위도,경도 추출

In [124]:
traffic_link['lon']=''
traffic_link['lat']=''
for i in range(len(traffic_link)):
    width =str(traffic_link['geometry'][i])[18:-2].split(',')
    traffic_link['lon'][i] = (eval(width[0].split(' ')[0])+eval(width[1].split(' ')[1]))/2
    traffic_link['lat'][i] = (eval(width[0].split(' ')[1])+eval(width[1].split(' ')[2]))/2

/opt/app-root/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/opt/app-root/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [125]:
traffic_link

link_id    시군구명 읍면동명  전체_추정교통량  승용차_추정교통량  버스_추정교통량  화물차_추정교통량  \
0      478353816  수원시권선구  금곡동     11690       8588       424       2678   
1      478353819  수원시권선구  금곡동     14564      10664       696       3203   
2      478466540  수원시권선구  입북동     17031      12607       604       3820   
3      478466541  수원시권선구  입북동     18025      13367       612       4046   
4      478466543  수원시권선구  입북동     19989      14868       762       4359   
...          ...     ...  ...       ...        ...       ...        ...   
11663  478442493  수원시장안구  파장동     16417      12142       446       3830   
11664  478442498  수원시장안구  파장동      3746       2831       104        812   
11665  478442497  수원시장안구  파장동     19727      14766       907       4054   
11666  478442264  수원시장안구  파장동     26583      19942      1047       5594   
11667  478442499  수원시장안구  파장동     19910      14936       784       4190   

      상세도로망_LinkID                                           geometry  \
0      47835381601  MULTILINESTRING ((126.94994 37.28007, 126.9499...   
1      47835381901  MULTILINESTRING ((126.94972 37.25199, 126.9496...   
2      47846654001  MULTILINESTRING ((126.94859 37.28954, 126.9486...   
3      47846654101  MULTILINESTRING ((126.94851 37.28987, 126.9485...   
4      47846654301  MULTILINESTRING ((126.94876 37.28956, 126.9486...   
...            ...                                                ...   
11663  47844249301  MULTILINESTRING ((126.99392 37.31739, 126.9936...   
11664  47844249801  MULTILINESTRING ((126.99114 37.31845, 126.9915...   
11665  47844249701  MULTILINESTRING ((126.99133 37.32058, 126.9912...   
11666  47844226401  MULTILINESTRING ((126.98800 37.32578, 126.9882...   
11667  47844249901  MULTILINESTRING ((126.99125 37.32040, 126.9912...   

           lon      lat  
0       126.95  37.2799  
1       126.95  37.2524  
2      126.949  37.2895  
3      126.949  37.2897  
4      126.949  37.2897  
...        ...      ...  
11663  126.994  37.3175  
11664  126.991  37.3183  
11665  126.991  37.3207  
11666  126.988  37.3256  
11667  126.991  37.3204  

[11668 rows x 11 columns]

다른 분석시 사용하기 위해 저장

In [126]:
traffic_link.to_csv('data/traffic_link.csv',index=False)
traffic_link.to_csv('data/traffic_link.txt',sep=',',index=False)

## 버스정류소 기준 반경 300m 이내 도로찾기

#### 방법 
Harversine 공식을 이용해 버스정류소의 좌표 기준 반경 300m이내의 도로를 모두 구한 후에, 반경 300m 이내 도로의 교통추정량 합을 버스정류장의 교통추정량으로 간주한다

버스정류장 전처리

In [127]:
bus = pd.read_csv('1.수원시_버스정류장.csv')
bus_raw = bus.copy() # 시각화를 위해 raw  data 남겨두기
bus

정류장ID     정류장명  정류장유형 환승역타입         위치(동)   쉘터  BIS설치여부  LED  LCD  \
0     201000313     권선구청  시내,마을    일반   경기도 수원시 서둔동  1.0      1.0  1.0  NaN   
1     201000344    중보들공원  시내,마을    일반    경기도 수원시 평동  1.0      NaN  NaN  NaN   
2     201000345  남수원초등학교     시내    일반  경기도 수원시 세류2동  NaN      1.0  NaN  NaN   
3     201000379   신미주아파트     시내    일반   경기도 수원시 금곡동  NaN      1.0  1.0  NaN   
4     201000055   농수산물시장     시내    일반  경기도 수원시 권선1동  NaN      1.0  NaN  1.0   
...         ...      ...    ...   ...           ...  ...      ...  ...  ...   
1174  201000266   구운초등학교     시내    일반   경기도 수원시 구운동  NaN      NaN  NaN  NaN   
1175  228000723    경희대정문  시내,마을    일반  경기도 수원시 영통1동  NaN      NaN  NaN  NaN   
1176  233001574   수영리검사소  시내,마을    일반    경기도 수원시 평동  NaN      NaN  NaN  NaN   
1177  201000123     자혜학교     시내    일반   경기도 수원시 서둔동  NaN      NaN  NaN  NaN   
1178  233001986   수영리검사소  시내,마을    일반    경기도 수원시 평동  NaN      NaN  NaN  NaN   

      LED+LCD복합형  알뜰형  임대형(음성)         lon        lat  
0            NaN  NaN      NaN  126.970983  37.256983  
1            NaN  NaN      NaN  126.991029  37.244605  
2            NaN  1.0      NaN  127.016000  37.251091  
3            NaN  NaN      NaN  126.953845  37.268806  
4            NaN  NaN      NaN  127.029815  37.256755  
...          ...  ...      ...         ...        ...  
1174         NaN  NaN      NaN  126.978712  37.281431  
1175         NaN  NaN      NaN  127.077907  37.247475  
1176         NaN  NaN      NaN  126.968503  37.232512  
1177         NaN  NaN      NaN  126.977880  37.265151  
1178         NaN  NaN      NaN  126.968669  37.232706  

[1179 rows x 14 columns]

분석 시, BIS 설치가 되어있지 않은 정류소는 제외하기로 함 -> BIS설치여부 1 아닌 행 삭제

In [128]:
bus.drop(bus[bus['BIS설치여부']!=1].index, inplace=True)
bus
bus.to_csv('data/bus_without_noBIS.csv',index=False) # BIS설치되어있지 않은 정류소 제거한 데이터 저장해놓기(다른 분석때 활용하기 위함)

In [129]:
bus.reset_index(inplace=True, drop=True)
bus

정류장ID                  정류장명  정류장유형 환승역타입         위치(동)   쉘터  BIS설치여부  \
0    201000313                  권선구청  시내,마을    일반   경기도 수원시 서둔동  1.0      1.0   
1    201000345               남수원초등학교     시내    일반  경기도 수원시 세류2동  NaN      1.0   
2    201000379                신미주아파트     시내    일반   경기도 수원시 금곡동  NaN      1.0   
3    201000055                농수산물시장     시내    일반  경기도 수원시 권선1동  NaN      1.0   
4    201000268                 곡반중학교     시내    일반   경기도 수원시 곡선동  1.0      1.0   
..         ...                   ...    ...   ...           ...  ...      ...   
923  202000249                 화서오거리     시내    일반  경기도 수원시 화서1동  NaN      1.0   
924  202000241  숙지공원.화서동굿모닝힐.화서다산도서관  시내,마을    일반  경기도 수원시 화서2동  NaN      1.0   
925  202000246           엘지아파트.금강아파트     시내    일반  경기도 수원시 화서2동  1.0      1.0   
926  202000239              꽃뫼신동아아파트  시내,마을    일반  경기도 수원시 화서2동  NaN      1.0   
927  202000238                 진흥아파트  시내,마을    일반  경기도 수원시 화서2동  1.0      1.0   

     LED  LCD  LED+LCD복합형  알뜰형  임대형(음성)         lon        lat  
0    1.0  NaN         NaN  NaN      NaN  126.970983  37.256983  
1    NaN  NaN         NaN  1.0      NaN  127.016000  37.251091  
2    1.0  NaN         NaN  NaN      NaN  126.953845  37.268806  
3    NaN  1.0         NaN  NaN      NaN  127.029815  37.256755  
4    1.0  NaN         NaN  NaN      NaN  127.034254  37.246081  
..   ...  ...         ...  ...      ...         ...        ...  
923  NaN  NaN         1.0  NaN      NaN  126.999697  37.278865  
924  NaN  1.0         NaN  NaN      NaN  126.995823  37.284316  
925  NaN  NaN         1.0  NaN      NaN  126.981714  37.287994  
926  NaN  1.0         NaN  NaN      NaN  126.986076  37.286663  
927  NaN  1.0         NaN  NaN      NaN  126.985096  37.289506  

[928 rows x 14 columns]

#### traffic_link 데이터의 도로 좌표와 버스정류소의 좌표를 배열로 만든 후,Haversine 거리 구하기
버스정류소 좌표 array 로 만들기

In [130]:
bus_array = bus[['정류장ID','lon','lat']]
bus_array['정류장ID'] = bus_array['정류장ID'].astype(str)
bus_array = bus_array[['정류장ID','lon','lat']].to_numpy()
bus_array

/opt/app-root/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


array([['201000313', 126.9709828, 37.25698318],
       ['201000345', 127.0160003, 37.25109066],
       ['201000379', 126.9538449, 37.26880555],
       ...,
       ['202000246', 126.98171380000001, 37.28799446],
       ['202000239', 126.98607630000001, 37.28666265],
       ['202000238', 126.9850962, 37.28950644]], dtype=object)

도로 좌표 array로 만들기

In [131]:
road_array = traffic_link[['link_id','lon','lat']]
road_array['link_id'] = road_array['link_id'].astype(str)
road_array = road_array.to_numpy()
road_array

/opt/app-root/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


array([['478353816', 126.94994300349765, 37.2799393291166],
       ['478353819', 126.9496581688401, 37.25241332921227],
       ['478466540', 126.94859898575524, 37.28947606122354],
       ...,
       ['478442497', 126.99128902337804, 37.32070560405552],
       ['478442264', 126.98811878682424, 37.325603249365244],
       ['478442499', 126.99126126832596, 37.32035479161647]], dtype=object)

데이터 길이 확인

In [132]:
print(bus_array.shape[0],road_array.shape[0])

928 11668


harversine 공식 이용하여 300m인 도로 추출하기

In [133]:
index_300m = [] 
for i in range(bus_array.shape[0]):
    each_road=[]
    for j in range(road_array.shape[0]):
        h_dis = GeoUtil.get_harversion_distance(bus_array[i,1], bus_array[i,2], road_array[j,1],road_array[j,2])
        if h_dis<=0.3:  
            each_road.append(road_array[j,0])
            
    index_300m.append(each_road) 

중복되는 도로 삭제

In [134]:
for i in range(len(index_300m)):
    index_300m[i] = list(set(index_300m[i]))

bus 데이터에 위에서 구한 도로 리스트를 컬럼으로 생성

In [135]:
bus['link_id'] = index_300m
bus.head()

정류장ID     정류장명  정류장유형 환승역타입         위치(동)   쉘터  BIS설치여부  LED  LCD  \
0  201000313     권선구청  시내,마을    일반   경기도 수원시 서둔동  1.0      1.0  1.0  NaN   
1  201000345  남수원초등학교     시내    일반  경기도 수원시 세류2동  NaN      1.0  NaN  NaN   
2  201000379   신미주아파트     시내    일반   경기도 수원시 금곡동  NaN      1.0  1.0  NaN   
3  201000055   농수산물시장     시내    일반  경기도 수원시 권선1동  NaN      1.0  NaN  1.0   
4  201000268    곡반중학교     시내    일반   경기도 수원시 곡선동  1.0      1.0  1.0  NaN   

   LED+LCD복합형  알뜰형  임대형(음성)         lon        lat  \
0         NaN  NaN      NaN  126.970983  37.256983   
1         NaN  1.0      NaN  127.016000  37.251091   
2         NaN  NaN      NaN  126.953845  37.268806   
3         NaN  NaN      NaN  127.029815  37.256755   
4         NaN  NaN      NaN  127.034254  37.246081   

                                             link_id  
0  [478452002, 478454556, 478450823, 478452290, 4...  
1  [571359236, 571301180, 571346126, 571352254, 5...  
2  [478448429, 478444899, 478448432, 478444888, 4...  
3  [571411027, 571410957, 571405249, 571411456, 5...  
4  [571345044, 571345043, 571369566, 571344574, 5...

리스트로 구성된 도로의 link id를 행으로 분리하기

In [136]:
for i in range(len(bus)):
    bus['link_id'][i] = ' '.join(bus['link_id'][i])
bus

/opt/app-root/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


정류장ID                  정류장명  정류장유형 환승역타입         위치(동)   쉘터  BIS설치여부  \
0    201000313                  권선구청  시내,마을    일반   경기도 수원시 서둔동  1.0      1.0   
1    201000345               남수원초등학교     시내    일반  경기도 수원시 세류2동  NaN      1.0   
2    201000379                신미주아파트     시내    일반   경기도 수원시 금곡동  NaN      1.0   
3    201000055                농수산물시장     시내    일반  경기도 수원시 권선1동  NaN      1.0   
4    201000268                 곡반중학교     시내    일반   경기도 수원시 곡선동  1.0      1.0   
..         ...                   ...    ...   ...           ...  ...      ...   
923  202000249                 화서오거리     시내    일반  경기도 수원시 화서1동  NaN      1.0   
924  202000241  숙지공원.화서동굿모닝힐.화서다산도서관  시내,마을    일반  경기도 수원시 화서2동  NaN      1.0   
925  202000246           엘지아파트.금강아파트     시내    일반  경기도 수원시 화서2동  1.0      1.0   
926  202000239              꽃뫼신동아아파트  시내,마을    일반  경기도 수원시 화서2동  NaN      1.0   
927  202000238                 진흥아파트  시내,마을    일반  경기도 수원시 화서2동  1.0      1.0   

     LED  LCD  LED+LCD복합형  알뜰형  임대형(음성)         lon        lat  \
0    1.0  NaN         NaN  NaN      NaN  126.970983  37.256983   
1    NaN  NaN         NaN  1.0      NaN  127.016000  37.251091   
2    1.0  NaN         NaN  NaN      NaN  126.953845  37.268806   
3    NaN  1.0         NaN  NaN      NaN  127.029815  37.256755   
4    1.0  NaN         NaN  NaN      NaN  127.034254  37.246081   
..   ...  ...         ...  ...      ...         ...        ...   
923  NaN  NaN         1.0  NaN      NaN  126.999697  37.278865   
924  NaN  1.0         NaN  NaN      NaN  126.995823  37.284316   
925  NaN  NaN         1.0  NaN      NaN  126.981714  37.287994   
926  NaN  1.0         NaN  NaN      NaN  126.986076  37.286663   
927  NaN  1.0         NaN  NaN      NaN  126.985096  37.289506   

                                               link_id  
0    478452002 478454556 478450823 478452290 478466...  
1    571359236 571301180 571346126 571352254 571371...  
2    478448429 478444899 478448432 478444888 478457...  
3    571411027 571410957 571405249 571411456 571411...  
4    571345044 571345043 571369566 571344574 571369...  
..                                                 ...  
923  478410623 571410650 571442423 478410497 571442...  
924  478444722 478444723 478451676 478457270 478450...  
925  478449090 478444109 478444117 478465378 478453...  
926  478444261 478444268 478449090 478444214 478461...  
927  478442390 478444165 478446861 478448099 478441...  

[928 rows x 15 columns]

traffic_link의 link_id str형식으로 변환하기

In [137]:
traffic_link['link_id'] = traffic_link['link_id'].astype(str)

In [138]:
split =bus.link_id.str.split(' ')
split

0      [478452002, 478454556, 478450823, 478452290, 4...
1      [571359236, 571301180, 571346126, 571352254, 5...
2      [478448429, 478444899, 478448432, 478444888, 4...
3      [571411027, 571410957, 571405249, 571411456, 5...
4      [571345044, 571345043, 571369566, 571344574, 5...
                             ...                        
923    [478410623, 571410650, 571442423, 478410497, 5...
924    [478444722, 478444723, 478451676, 478457270, 4...
925    [478449090, 478444109, 478444117, 478465378, 4...
926    [478444261, 478444268, 478449090, 478444214, 4...
927    [478442390, 478444165, 478446861, 478448099, 4...
Name: link_id, Length: 928, dtype: object

In [139]:
split = split.apply(lambda x: pd.Series(x))
split

0          1          2          3          4          5   \
0    478452002  478454556  478450823  478452290  478466691  478449676   
1    571359236  571301180  571346126  571352254  571371625  571359232   
2    478448429  478444899  478448432  478444888  478457302  478448437   
3    571411027  571410957  571405249  571411456  571411114  571457882   
4    571345044  571345043  571369566  571344574  571369568  571369572   
..         ...        ...        ...        ...        ...        ...   
923  478410623  571410650  571442423  478410497  571442349  571410708   
924  478444722  478444723  478451676  478457270  478450414  478448135   
925  478449090  478444109  478444117  478465378  478453796  478453798   
926  478444261  478444268  478449090  478444214  478461490  478445984   
927  478442390  478444165  478446861  478448099  478441972  478465378   

            6          7          8          9   ...   65   66   67   68   69  \
0    478466688  478466703  478466401  478464404  ...  NaN  NaN  NaN  NaN  NaN   
1    571351602  571346163  571310303  571359238  ...  NaN  NaN  NaN  NaN  NaN   
2    478463084  478457298  478463082  478448435  ...  NaN  NaN  NaN  NaN  NaN   
3    571441826  571411006  571405120  571405173  ...  NaN  NaN  NaN  NaN  NaN   
4    571351028  571365004  571350770  571359618  ...  NaN  NaN  NaN  NaN  NaN   
..         ...        ...        ...        ...  ...  ...  ...  ...  ...  ...   
923  571410699  571410304  571410238        NaN  ...  NaN  NaN  NaN  NaN  NaN   
924  478457268  478410686  478450416  478452804  ...  NaN  NaN  NaN  NaN  NaN   
925  478444103  478451215  478449768  478465376  ...  NaN  NaN  NaN  NaN  NaN   
926  478456668  478448098  478444269  478461480  ...  NaN  NaN  NaN  NaN  NaN   
927  478448098  478446473  478444152  478465374  ...  NaN  NaN  NaN  NaN  NaN   

      70   71   72   73   74  
0    NaN  NaN  NaN  NaN  NaN  
1    NaN  NaN  NaN  NaN  NaN  
2    NaN  NaN  NaN  NaN  NaN  
3    NaN  NaN  NaN  NaN  NaN  
4    NaN  NaN  NaN  NaN  NaN  
..   ...  ...  ...  ...  ...  
923  NaN  NaN  NaN  NaN  NaN  
924  NaN  NaN  NaN  NaN  NaN  
925  NaN  NaN  NaN  NaN  NaN  
926  NaN  NaN  NaN  NaN  NaN  
927  NaN  NaN  NaN  NaN  NaN  

[928 rows x 75 columns]

In [140]:
split.stack().reset_index(level=1, drop=True)

0      478452002
0      478454556
0      478450823
0      478452290
0      478466691
         ...    
927    478401578
927    478444153
927    478446257
927    478446298
927    478446477
Length: 25758, dtype: object

In [141]:
split = split.stack().reset_index(level=1, drop=True).to_frame('link_id_2')
split

link_id_2
0    478452002
0    478454556
0    478450823
0    478452290
0    478466691
..         ...
927  478401578
927  478444153
927  478446257
927  478446298
927  478446477

[25758 rows x 1 columns]

In [142]:
bus = bus.merge(split, left_index=True, right_index=True, how='left')
bus.head()

정류장ID  정류장명  정류장유형 환승역타입        위치(동)   쉘터  BIS설치여부  LED  LCD  \
0  201000313  권선구청  시내,마을    일반  경기도 수원시 서둔동  1.0      1.0  1.0  NaN   
0  201000313  권선구청  시내,마을    일반  경기도 수원시 서둔동  1.0      1.0  1.0  NaN   
0  201000313  권선구청  시내,마을    일반  경기도 수원시 서둔동  1.0      1.0  1.0  NaN   
0  201000313  권선구청  시내,마을    일반  경기도 수원시 서둔동  1.0      1.0  1.0  NaN   
0  201000313  권선구청  시내,마을    일반  경기도 수원시 서둔동  1.0      1.0  1.0  NaN   

   LED+LCD복합형  알뜰형  임대형(음성)         lon        lat  \
0         NaN  NaN      NaN  126.970983  37.256983   
0         NaN  NaN      NaN  126.970983  37.256983   
0         NaN  NaN      NaN  126.970983  37.256983   
0         NaN  NaN      NaN  126.970983  37.256983   
0         NaN  NaN      NaN  126.970983  37.256983   

                                             link_id  link_id_2  
0  478452002 478454556 478450823 478452290 478466...  478452002  
0  478452002 478454556 478450823 478452290 478466...  478454556  
0  478452002 478454556 478450823 478452290 478466...  478450823  
0  478452002 478454556 478450823 478452290 478466...  478452290  
0  478452002 478454556 478450823 478452290 478466...  478466691

link_id feature 삭제하기

In [143]:
del bus['link_id']

In [144]:
bus.rename(columns={'link_id_2':'link_id'},inplace=True)
bus.head()

정류장ID  정류장명  정류장유형 환승역타입        위치(동)   쉘터  BIS설치여부  LED  LCD  \
0  201000313  권선구청  시내,마을    일반  경기도 수원시 서둔동  1.0      1.0  1.0  NaN   
0  201000313  권선구청  시내,마을    일반  경기도 수원시 서둔동  1.0      1.0  1.0  NaN   
0  201000313  권선구청  시내,마을    일반  경기도 수원시 서둔동  1.0      1.0  1.0  NaN   
0  201000313  권선구청  시내,마을    일반  경기도 수원시 서둔동  1.0      1.0  1.0  NaN   
0  201000313  권선구청  시내,마을    일반  경기도 수원시 서둔동  1.0      1.0  1.0  NaN   

   LED+LCD복합형  알뜰형  임대형(음성)         lon        lat    link_id  
0         NaN  NaN      NaN  126.970983  37.256983  478452002  
0         NaN  NaN      NaN  126.970983  37.256983  478454556  
0         NaN  NaN      NaN  126.970983  37.256983  478450823  
0         NaN  NaN      NaN  126.970983  37.256983  478452290  
0         NaN  NaN      NaN  126.970983  37.256983  478466691

#### 버스정류소와 교통량 합치기
link_id feature를 str형식으로 변환

In [145]:
bus['link_id'] = bus['link_id'].astype(str)
traffic_link['link_id'] = traffic_link['link_id'].astype(str)

두 변수 합치기

In [146]:
bus_with_traffic = pd.merge(bus,traffic_link,how='left',on=('link_id'))
bus_with_traffic.head()

정류장ID  정류장명  정류장유형 환승역타입        위치(동)   쉘터  BIS설치여부  LED  LCD  \
0  201000313  권선구청  시내,마을    일반  경기도 수원시 서둔동  1.0      1.0  1.0  NaN   
1  201000313  권선구청  시내,마을    일반  경기도 수원시 서둔동  1.0      1.0  1.0  NaN   
2  201000313  권선구청  시내,마을    일반  경기도 수원시 서둔동  1.0      1.0  1.0  NaN   
3  201000313  권선구청  시내,마을    일반  경기도 수원시 서둔동  1.0      1.0  1.0  NaN   
4  201000313  권선구청  시내,마을    일반  경기도 수원시 서둔동  1.0      1.0  1.0  NaN   

   LED+LCD복합형  ...    시군구명  읍면동명  전체_추정교통량  승용차_추정교통량 버스_추정교통량 화물차_추정교통량  \
0         NaN  ...  수원시권선구   서둔동    1995.0     1801.0      6.0     189.0   
1         NaN  ...  수원시권선구   서둔동    2203.0     1954.0      8.0     242.0   
2         NaN  ...  수원시권선구   서둔동    1576.0     1341.0      3.0     232.0   
3         NaN  ...  수원시권선구   서둔동     152.0      112.0      0.0      41.0   
4         NaN  ...  수원시권선구   서둔동     229.0      169.0      0.0      60.0   

  상세도로망_LinkID                                           geometry    lon_y  \
0  47845200202  MULTILINESTRING ((126.97062 37.25893, 126.9703...   126.97   
1  47845200201  MULTILINESTRING ((126.97062 37.25893, 126.9703...   126.97   
2  47845455601  MULTILINESTRING ((126.97228 37.25917, 126.9729...  126.973   
3  47845082301  MULTILINESTRING ((126.96843 37.25679, 126.9687...  126.969   
4  47845082302  MULTILINESTRING ((126.96843 37.25679, 126.9687...  126.969   

     lat_y  
0  37.2596  
1  37.2596  
2  37.2592  
3  37.2568  
4  37.2568  

[5 rows x 25 columns]

In [147]:
bus_with_traffic.shape

(45251, 25)

필요한 변수만 뽑아내기

In [148]:
bus_with_traffic = bus_with_traffic[['정류장ID','정류장명','시군구명','읍면동명','전체_추정교통량','승용차_추정교통량','버스_추정교통량','화물차_추정교통량','상세도로망_LinkID','link_id','geometry','lon_y','lat_y']]
bus_with_traffic[:10]

정류장ID  정류장명    시군구명 읍면동명  전체_추정교통량  승용차_추정교통량  버스_추정교통량  화물차_추정교통량  \
0  201000313  권선구청  수원시권선구  서둔동    1995.0     1801.0       6.0      189.0   
1  201000313  권선구청  수원시권선구  서둔동    2203.0     1954.0       8.0      242.0   
2  201000313  권선구청  수원시권선구  서둔동    1576.0     1341.0       3.0      232.0   
3  201000313  권선구청  수원시권선구  서둔동     152.0      112.0       0.0       41.0   
4  201000313  권선구청  수원시권선구  서둔동     229.0      169.0       0.0       60.0   
5  201000313  권선구청  수원시권선구  서둔동    2163.0     1433.0       8.0      722.0   
6  201000313  권선구청  수원시권선구  서둔동    2236.0     1494.0       9.0      733.0   
7  201000313  권선구청  수원시권선구   평동     234.0      178.0       0.0       56.0   
8  201000313  권선구청  수원시권선구   평동     542.0      483.0       1.0       59.0   
9  201000313  권선구청  수원시권선구  서둔동     206.0       91.0       0.0      114.0   

  상세도로망_LinkID    link_id                                           geometry  \
0  47845200202  478452002  MULTILINESTRING ((126.97062 37.25893, 126.9703...   
1  47845200201  478452002  MULTILINESTRING ((126.97062 37.25893, 126.9703...   
2  47845455601  478454556  MULTILINESTRING ((126.97228 37.25917, 126.9729...   
3  47845082301  478450823  MULTILINESTRING ((126.96843 37.25679, 126.9687...   
4  47845082302  478450823  MULTILINESTRING ((126.96843 37.25679, 126.9687...   
5  47845229002  478452290  MULTILINESTRING ((126.96843 37.25679, 126.9686...   
6  47845229001  478452290  MULTILINESTRING ((126.96843 37.25679, 126.9686...   
7  47846669102  478466691  MULTILINESTRING ((126.97136 37.25567, 126.9704...   
8  47846669101  478466691  MULTILINESTRING ((126.97136 37.25567, 126.9704...   
9  47844967602  478449676  MULTILINESTRING ((126.97085 37.25784, 126.9714...   

     lon_y    lat_y  
0   126.97  37.2596  
1   126.97  37.2596  
2  126.973  37.2592  
3  126.969  37.2568  
4  126.969  37.2568  
5  126.969  37.2563  
6  126.969  37.2563  
7  126.971  37.2556  
8  126.971  37.2556  
9  126.971  37.2579

In [149]:
bus_with_traffic['link_id']=bus_with_traffic['link_id'].astype(str)
bus_with_traffic['상세도로망_LinkID']=bus_with_traffic['상세도로망_LinkID'].astype(str)

그룹화해서 sum 할 변수만 선택

In [150]:
bus_group = bus_with_traffic[['정류장ID','전체_추정교통량','승용차_추정교통량','버스_추정교통량','화물차_추정교통량']]
bus_group.head()

정류장ID  전체_추정교통량  승용차_추정교통량  버스_추정교통량  화물차_추정교통량
0  201000313    1995.0     1801.0       6.0      189.0
1  201000313    2203.0     1954.0       8.0      242.0
2  201000313    1576.0     1341.0       3.0      232.0
3  201000313     152.0      112.0       0.0       41.0
4  201000313     229.0      169.0       0.0       60.0

정류장id로 그룹화하기

In [151]:
bus_group=bus_group.groupby(['정류장ID'], as_index=False).sum()
bus_group

정류장ID  전체_추정교통량  승용차_추정교통량  버스_추정교통량  화물차_추정교통량
0    200000005  138507.0   107567.0     698.0    30243.0
1    200000006  144981.0   114321.0     738.0    29921.0
2    200000007  136500.0   105852.0     693.0    29957.0
3    200000008       0.0        0.0       0.0        0.0
4    200000009  538641.0   402702.0   14511.0   121426.0
..         ...       ...        ...       ...        ...
923  203000424  486849.0   401579.0    8970.0    76298.0
924  203000438  397516.0   327057.0    5089.0    65369.0
925  203000439  395725.0   325472.0    5086.0    65166.0
926  203000449  112658.0    90981.0     956.0    20718.0
927  203000450  536921.0   453662.0    5487.0    77775.0

[928 rows x 5 columns]

sum 하지 않은 변수들 중 중복값 삭제

In [152]:
unique_id = bus_with_traffic[['정류장ID','정류장명','시군구명','읍면동명','link_id']].drop_duplicates() 
unique_id

정류장ID   정류장명    시군구명  읍면동명    link_id
0      201000313   권선구청  수원시권선구   서둔동  478452002
2      201000313   권선구청  수원시권선구   서둔동  478454556
3      201000313   권선구청  수원시권선구   서둔동  478450823
5      201000313   권선구청  수원시권선구   서둔동  478452290
7      201000313   권선구청  수원시권선구    평동  478466691
...          ...    ...     ...   ...        ...
45245  202000238  진흥아파트  수원시팔달구  화서2동  478401578
45247  202000238  진흥아파트  수원시장안구  정자3동  478444153
45248  202000238  진흥아파트  수원시장안구  정자3동  478446257
45249  202000238  진흥아파트  수원시장안구  정자3동  478446298
45250  202000238  진흥아파트  수원시장안구  정자3동  478446477

[25758 rows x 5 columns]

sum 결과와 '정류장ID','정류장명','시군구명','읍면동명','link_id' merge

In [153]:
busTraffic_sum = pd.merge(bus_group,unique_id, how='left',on='정류장ID')
busTraffic_sum

정류장ID  전체_추정교통량  승용차_추정교통량  버스_추정교통량  화물차_추정교통량            정류장명  \
0      200000005  138507.0   107567.0     698.0    30243.0  광교산입구.경기대수원캠퍼스   
1      200000005  138507.0   107567.0     698.0    30243.0  광교산입구.경기대수원캠퍼스   
2      200000005  138507.0   107567.0     698.0    30243.0  광교산입구.경기대수원캠퍼스   
3      200000005  138507.0   107567.0     698.0    30243.0  광교산입구.경기대수원캠퍼스   
4      200000005  138507.0   107567.0     698.0    30243.0  광교산입구.경기대수원캠퍼스   
...          ...       ...        ...       ...        ...             ...   
25753  203000450  536921.0   453662.0    5487.0    77775.0       살구골.서광아파트   
25754  203000450  536921.0   453662.0    5487.0    77775.0       살구골.서광아파트   
25755  203000450  536921.0   453662.0    5487.0    77775.0       살구골.서광아파트   
25756  203000450  536921.0   453662.0    5487.0    77775.0       살구골.서광아파트   
25757  203000450  536921.0   453662.0    5487.0    77775.0       살구골.서광아파트   

         시군구명  읍면동명    link_id  
0      수원시영통구  광교1동  571455442  
1      수원시장안구   연무동  571458061  
2      수원시장안구   연무동  571458057  
3      수원시장안구   연무동  571455447  
4      수원시장안구   연무동  571455448  
...       ...   ...        ...  
25753  수원시영통구  영통3동  571310177  
25754  수원시영통구  영통3동  571301020  
25755  수원시영통구  영통3동  571351171  
25756  수원시영통구  영통3동  571310181  
25757  수원시영통구  영통2동  571349222  

[25758 rows x 9 columns]

이후 전처리를 위한 데이터 분리 및 저장

In [154]:
busTraffic_final = busTraffic_sum.copy()
busTraffic_final['화물차_추정교통량_x40'] = busTraffic_final['화물차_추정교통량']*40
busTraffic_final['일반차_추정교통량'] = busTraffic_final['승용차_추정교통량']+ busTraffic_final['버스_추정교통량']
busTraffic_final = busTraffic_final[['정류장ID','일반차_추정교통량','화물차_추정교통량_x40']]
busTraffic_final 

정류장ID  일반차_추정교통량  화물차_추정교통량_x40
0      200000005   108265.0      1209720.0
1      200000005   108265.0      1209720.0
2      200000005   108265.0      1209720.0
3      200000005   108265.0      1209720.0
4      200000005   108265.0      1209720.0
...          ...        ...            ...
25753  203000450   459149.0      3111000.0
25754  203000450   459149.0      3111000.0
25755  203000450   459149.0      3111000.0
25756  203000450   459149.0      3111000.0
25757  203000450   459149.0      3111000.0

[25758 rows x 3 columns]

In [155]:
busTraffic_final=busTraffic_final.drop_duplicates() 
busTraffic_final.reset_index(drop=True,inplace=True)
busTraffic_final.to_csv('data/busTraffic_final.txt',sep=',',index=False)

In [156]:
busTraffic_final

정류장ID  일반차_추정교통량  화물차_추정교통량_x40
0    200000005   108265.0      1209720.0
1    200000006   115059.0      1196840.0
2    200000007   106545.0      1198280.0
3    200000008        0.0            0.0
4    200000009   417213.0      4857040.0
..         ...        ...            ...
923  203000424   410549.0      3051920.0
924  203000438   332146.0      2614760.0
925  203000439   330558.0      2606640.0
926  203000449    91937.0       828720.0
927  203000450   459149.0      3111000.0

[928 rows x 3 columns]

### 분석 및 시각화
##### busTraffic_sum : 버스정류장 좌표 + 반경300m 이내 도로 좌표
##### busTraffic_only_sum : 버스정류장 좌표 (도로 정보 미포함)

In [157]:
busTraffic_only_sum = busTraffic_sum[['정류장ID','정류장명','전체_추정교통량','승용차_추정교통량','버스_추정교통량','화물차_추정교통량']]
busTraffic_only_sum=busTraffic_only_sum.drop_duplicates() 
busTraffic_only_sum

정류장ID                  정류장명  전체_추정교통량  승용차_추정교통량  버스_추정교통량  \
0      200000005        광교산입구.경기대수원캠퍼스  138507.0   107567.0     698.0   
22     200000006  광교공원.경기대수원캠퍼스입구.연무시장  144981.0   114321.0     738.0   
46     200000007        광교산입구.경기대수원캠퍼스  136500.0   105852.0     693.0   
67     200000008                   문암골       0.0        0.0       0.0   
68     200000009                비석거리입구  538641.0   402702.0   14511.0   
...          ...                   ...       ...        ...       ...   
25624  203000424           수원시도시안전통합센터  486849.0   401579.0    8970.0   
25665  203000438              광교카페거리입구  397516.0   327057.0    5089.0   
25691  203000439              광교카페거리입구  395725.0   325472.0    5086.0   
25716  203000449      대선초.e편한세상영통2차아파트  112658.0    90981.0     956.0   
25738  203000450             살구골.서광아파트  536921.0   453662.0    5487.0   

       화물차_추정교통량  
0        30243.0  
22       29921.0  
46       29957.0  
67           0.0  
68      121426.0  
...          ...  
25624    76298.0  
25665    65369.0  
25691    65166.0  
25716    20718.0  
25738    77775.0  

[928 rows x 6 columns]

In [158]:
busTraffic_only_sum.to_csv('data/busTraffic_only_sum.txt',sep=',',index=False)

인덱스 리셋하기

In [159]:
busTraffic_only_sum.reset_index(inplace=True, drop=True)
busTraffic_sum.reset_index(inplace=True, drop=True)

도로 좌표를 시각화하기위해 linkid 데이터의 geometry 열을 추가

In [160]:
busTraffic_sum = pd.merge(busTraffic_sum,linkid, how='left',on=['link_id'])
busTraffic_sum.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25758 entries, 0 to 25757
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   정류장ID      25758 non-null  int64   
 1   전체_추정교통량   25758 non-null  float64 
 2   승용차_추정교통량  25758 non-null  float64 
 3   버스_추정교통량   25758 non-null  float64 
 4   화물차_추정교통량  25758 non-null  float64 
 5   정류장명       25758 non-null  object  
 6   시군구명       25753 non-null  object  
 7   읍면동명       25753 non-null  object  
 8   link_id    25758 non-null  object  
 9   geometry   25753 non-null  geometry
dtypes: float64(4), geometry(1), int64(1), object(4)
memory usage: 2.2+ MB


In [161]:
busTraffic_sum.head()

정류장ID  전체_추정교통량  승용차_추정교통량  버스_추정교통량  화물차_추정교통량            정류장명  \
0  200000005  138507.0   107567.0     698.0    30243.0  광교산입구.경기대수원캠퍼스   
1  200000005  138507.0   107567.0     698.0    30243.0  광교산입구.경기대수원캠퍼스   
2  200000005  138507.0   107567.0     698.0    30243.0  광교산입구.경기대수원캠퍼스   
3  200000005  138507.0   107567.0     698.0    30243.0  광교산입구.경기대수원캠퍼스   
4  200000005  138507.0   107567.0     698.0    30243.0  광교산입구.경기대수원캠퍼스   

     시군구명  읍면동명    link_id                                           geometry  
0  수원시영통구  광교1동  571455442  MULTILINESTRING ((127.03373 37.30163, 127.0337...  
1  수원시장안구   연무동  571458061  MULTILINESTRING ((127.03204 37.30233, 127.0320...  
2  수원시장안구   연무동  571458057  MULTILINESTRING ((127.02967 37.29983, 127.0295...  
3  수원시장안구   연무동  571455447  MULTILINESTRING ((127.03178 37.30108, 127.0336...  
4  수원시장안구   연무동  571455448  MULTILINESTRING ((127.03105 37.30037, 127.0308...

In [162]:
busTraffic_only_sum.head()

정류장ID                  정류장명  전체_추정교통량  승용차_추정교통량  버스_추정교통량  화물차_추정교통량
0  200000005        광교산입구.경기대수원캠퍼스  138507.0   107567.0     698.0    30243.0
1  200000006  광교공원.경기대수원캠퍼스입구.연무시장  144981.0   114321.0     738.0    29921.0
2  200000007        광교산입구.경기대수원캠퍼스  136500.0   105852.0     693.0    29957.0
3  200000008                   문암골       0.0        0.0       0.0        0.0
4  200000009                비석거리입구  538641.0   402702.0   14511.0   121426.0

시각화를 위해 버스정류장 좌표와 merge 후 중복 제거하고 인덱스 리셋

In [163]:
busTraffic_sum = pd.merge(busTraffic_sum,bus[['정류장ID','lon','lat']], how='left',on='정류장ID').drop_duplicates()
busTraffic_only_sum = pd.merge(busTraffic_only_sum,bus[['정류장ID','lon','lat']], how='left',on='정류장ID').drop_duplicates()

In [164]:
busTraffic_sum.reset_index(drop=True,inplace=True)
busTraffic_only_sum.reset_index(drop=True,inplace=True)

#### 전체 교통량 가장 많은 버스정류소 상위 30개

In [165]:
total_top30 = busTraffic_only_sum.sort_values(by=["전체_추정교통량"], ascending=[False])[:30]
total_top30.head(10)

정류장ID                 정류장명   전체_추정교통량  승용차_추정교통량  버스_추정교통량  \
893  203000339              수원광교박물관  1391413.0  1120371.0   24087.0   
894  203000340              수원광교박물관  1391413.0  1120371.0   24087.0   
688  203000041  차세대융합기술연구원.푸르지오월드마크  1382606.0  1111247.0   21188.0   
240  201000059               가게앞.평동  1315615.0  1083349.0   17110.0   
286  201000116             서수원버스터미널  1225898.0   987174.0   16381.0   
822  203000236             디지털엠파이어2  1165199.0   955298.0   11890.0   
366  201000229             농수산물유통센터  1134983.0   913393.0   16515.0   
360  201000211                우남아파트  1115844.0   923268.0   15411.0   
278  201000105           서수원터미널.이마트  1109385.0   894454.0   15952.0   
213  201000029             서수원버스터미널  1108732.0   890761.0   15357.0   

     화물차_추정교통량         lon        lat  
893   246961.0  127.049579  37.296470  
894   246961.0  127.049332  37.296417  
688   250182.0  127.047881  37.293429  
240   215159.0  126.995713  37.260709  
286   222344.0  126.970863  37.282907  
822   198008.0  127.049074  37.246861  
366   205076.0  126.968974  37.282895  
360   177174.0  127.033144  37.247750  
278   198982.0  126.969462  37.282781  
213   202620.0  126.970796  37.283312

#### 전체 교통량 상위 30개 버스정류소 + 반경 300m 이내 도로

In [166]:
total_top30_road = pd.merge(total_top30,busTraffic_sum[['정류장ID','geometry']],how='left',on=('정류장ID'))
total_top30_road

정류장ID      정류장명   전체_추정교통량  승용차_추정교통량  버스_추정교통량  화물차_추정교통량  \
0     203000339   수원광교박물관  1391413.0  1120371.0   24087.0   246961.0   
1     203000339   수원광교박물관  1391413.0  1120371.0   24087.0   246961.0   
2     203000339   수원광교박물관  1391413.0  1120371.0   24087.0   246961.0   
3     203000339   수원광교박물관  1391413.0  1120371.0   24087.0   246961.0   
4     203000339   수원광교박물관  1391413.0  1120371.0   24087.0   246961.0   
...         ...       ...        ...        ...       ...        ...   
1392  203000064  e편한세상아파트   861846.0   731014.0   13245.0   117591.0   
1393  203000064  e편한세상아파트   861846.0   731014.0   13245.0   117591.0   
1394  203000064  e편한세상아파트   861846.0   731014.0   13245.0   117591.0   
1395  203000064  e편한세상아파트   861846.0   731014.0   13245.0   117591.0   
1396  203000064  e편한세상아파트   861846.0   731014.0   13245.0   117591.0   

             lon        lat                                           geometry  
0     127.049579  37.296470  MULTILINESTRING ((127.04906 37.29521, 127.0492...  
1     127.049579  37.296470  MULTILINESTRING ((127.04917 37.29565, 127.0494...  
2     127.049579  37.296470  MULTILINESTRING ((127.04967 37.29736, 127.0492...  
3     127.049579  37.296470  MULTILINESTRING ((127.05042 37.29500, 127.0511...  
4     127.049579  37.296470  MULTILINESTRING ((127.04912 37.29836, 127.0475...  
...          ...        ...                                                ...  
1392  127.067199  37.246392  MULTILINESTRING ((127.06692 37.24424, 127.0665...  
1393  127.067199  37.246392  MULTILINESTRING ((127.06492 37.24495, 127.0644...  
1394  127.067199  37.246392  MULTILINESTRING ((127.06715 37.24641, 127.0694...  
1395  127.067199  37.246392  MULTILINESTRING ((127.06644 37.24636, 127.0671...  
1396  127.067199  37.246392  MULTILINESTRING ((127.06710 37.24709, 127.0666...  

[1397 rows x 9 columns]

#### 화물차 교통량 가장 많은 버스정류소 상위 30개화물차 교통량 가장 많은 버스정류소 상위 30개

In [167]:
truck_top30 = busTraffic_only_sum.sort_values(by=["화물차_추정교통량"], ascending=[False])[:30]
truck_top30.head(10)

정류장ID                 정류장명   전체_추정교통량  승용차_추정교통량  버스_추정교통량  \
688  203000041  차세대융합기술연구원.푸르지오월드마크  1382606.0  1111247.0   21188.0   
894  203000340              수원광교박물관  1391413.0  1120371.0   24087.0   
893  203000339              수원광교박물관  1391413.0  1120371.0   24087.0   
286  201000116             서수원버스터미널  1225898.0   987174.0   16381.0   
240  201000059               가게앞.평동  1315615.0  1083349.0   17110.0   
366  201000229             농수산물유통센터  1134983.0   913393.0   16515.0   
213  201000029             서수원버스터미널  1108732.0   890761.0   15357.0   
287  201000117                구운사거리  1089670.0   871048.0   16796.0   
220  201000036       구운사거리.농수산물유통센터  1089670.0   871048.0   16796.0   
278  201000105           서수원터미널.이마트  1109385.0   894454.0   15952.0   

     화물차_추정교통량         lon        lat  
688   250182.0  127.047881  37.293429  
894   246961.0  127.049332  37.296417  
893   246961.0  127.049579  37.296470  
286   222344.0  126.970863  37.282907  
240   215159.0  126.995713  37.260709  
366   205076.0  126.968974  37.282895  
213   202620.0  126.970796  37.283312  
287   201831.0  126.969284  37.285438  
220   201831.0  126.968596  37.284879  
278   198982.0  126.969462  37.282781

#### 화물 교통량 상위 30개 버스정류소 + 반경 300m 이내도로

In [168]:
truck_top30_road = pd.merge(truck_top30,busTraffic_sum[['정류장ID','geometry']],how='left',on=('정류장ID'))
truck_top30_road

정류장ID                 정류장명   전체_추정교통량  승용차_추정교통량  버스_추정교통량  \
0     203000041  차세대융합기술연구원.푸르지오월드마크  1382606.0  1111247.0   21188.0   
1     203000041  차세대융합기술연구원.푸르지오월드마크  1382606.0  1111247.0   21188.0   
2     203000041  차세대융합기술연구원.푸르지오월드마크  1382606.0  1111247.0   21188.0   
3     203000041  차세대융합기술연구원.푸르지오월드마크  1382606.0  1111247.0   21188.0   
4     203000041  차세대융합기술연구원.푸르지오월드마크  1382606.0  1111247.0   21188.0   
...         ...                  ...        ...        ...       ...   
1367  203000335           원천레이크파크아파트   968929.0   806709.0   12376.0   
1368  203000335           원천레이크파크아파트   968929.0   806709.0   12376.0   
1369  203000335           원천레이크파크아파트   968929.0   806709.0   12376.0   
1370  203000335           원천레이크파크아파트   968929.0   806709.0   12376.0   
1371  203000335           원천레이크파크아파트   968929.0   806709.0   12376.0   

      화물차_추정교통량         lon        lat  \
0      250182.0  127.047881  37.293429   
1      250182.0  127.047881  37.293429   
2      250182.0  127.047881  37.293429   
3      250182.0  127.047881  37.293429   
4      250182.0  127.047881  37.293429   
...         ...         ...        ...   
1367   149838.0  127.063215  37.268594   
1368   149838.0  127.063215  37.268594   
1369   149838.0  127.063215  37.268594   
1370   149838.0  127.063215  37.268594   
1371   149838.0  127.063215  37.268594   

                                               geometry  
0     MULTILINESTRING ((127.04906 37.29521, 127.0492...  
1     MULTILINESTRING ((127.05027 37.29206, 127.0505...  
2     MULTILINESTRING ((127.05027 37.29206, 127.0498...  
3     MULTILINESTRING ((127.05020 37.29503, 127.0504...  
4     MULTILINESTRING ((127.04905 37.29230, 127.0486...  
...                                                 ...  
1367  MULTILINESTRING ((127.06085 37.26987, 127.0612...  
1368  MULTILINESTRING ((127.06556 37.26667, 127.0654...  
1369  MULTILINESTRING ((127.06447 37.26625, 127.0645...  
1370  MULTILINESTRING ((127.06128 37.27008, 127.0610...  
1371  MULTILINESTRING ((127.06153 37.27051, 127.0615...  

[1372 rows x 9 columns]

#### 전체 교통량이 가장 적은 버스정류소 30개

In [169]:
total_bottom30 = busTraffic_only_sum.sort_values(by=["전체_추정교통량"])[:30]
total_bottom30.head(10)

정류장ID            정류장명  전체_추정교통량  승용차_추정교통량  버스_추정교통량  화물차_추정교통량  \
407  201000338      칠보산입구.자목마을       0.0        0.0       0.0        0.0   
697  203000051        광교중흥S클래스       0.0        0.0       0.0        0.0   
3    200000008             문암골       0.0        0.0       0.0        0.0   
333  201000182         당수동시민농장       0.0        0.0       0.0        0.0   
670  203000020        광교중흥S클래스       0.0        0.0       0.0        0.0   
387  201000260         국립축산과학원     608.0      589.0       0.0       18.0   
225  201000044             신곡동    1439.0     1343.0       2.0       94.0   
295  201000134           탑동사거리   15262.0    11770.0      68.0     3423.0   
300  201000139            탑동입구   15262.0    11770.0      68.0     3423.0   
902  203000352  광교에일린의뜰.광교호수공원   17467.0    14936.0     112.0     2419.0   

            lon        lat  
407  126.942783  37.263492  
697  127.059403  37.281158  
3    127.027700  37.308946  
333  126.940409  37.284237  
670  127.059162  37.281190  
387  126.966306  37.236135  
225  127.012492  37.249209  
295  126.979783  37.268949  
300  126.979922  37.269023  
902  127.059965  37.278855

#### 전체 교통량 하위 30개 버스정류소 + 반경 300m이내 도로

In [170]:
total_bottom30_road = pd.merge(total_bottom30,busTraffic_sum[['정류장ID','geometry']],how='left',on=('정류장ID'))
total_bottom30_road = total_bottom30_road [5:] # 하위 5개는 매칭된 도로가 없으므로 결측값 -> 시각화를 위해 geometry 정보 생략
total_bottom30_road 

정류장ID      정류장명  전체_추정교통량  승용차_추정교통량  버스_추정교통량  화물차_추정교통량  \
5    201000260   국립축산과학원     608.0      589.0       0.0       18.0   
6    201000044       신곡동    1439.0     1343.0       2.0       94.0   
7    201000134     탑동사거리   15262.0    11770.0      68.0     3423.0   
8    201000134     탑동사거리   15262.0    11770.0      68.0     3423.0   
9    201000139      탑동입구   15262.0    11770.0      68.0     3423.0   
..         ...       ...       ...        ...       ...        ...   
139  201000090  수원농협권선지점   48885.0    42631.0     237.0     6014.0   
140  201000090  수원농협권선지점   48885.0    42631.0     237.0     6014.0   
141  201000090  수원농협권선지점   48885.0    42631.0     237.0     6014.0   
142  201000090  수원농협권선지점   48885.0    42631.0     237.0     6014.0   
143  201000090  수원농협권선지점   48885.0    42631.0     237.0     6014.0   

            lon        lat                                           geometry  
5    126.966306  37.236135  MULTILINESTRING ((126.96583 37.23626, 126.9659...  
6    127.012492  37.249209  MULTILINESTRING ((127.01323 37.24785, 127.0137...  
7    126.979783  37.268949  MULTILINESTRING ((126.97956 37.26966, 126.9795...  
8    126.979783  37.268949  MULTILINESTRING ((126.98112 37.26643, 126.9806...  
9    126.979922  37.269023  MULTILINESTRING ((126.97956 37.26966, 126.9795...  
..          ...        ...                                                ...  
139  127.017519  37.258167  MULTILINESTRING ((127.02036 37.25665, 127.0205...  
140  127.017519  37.258167  MULTILINESTRING ((127.02036 37.25665, 127.0193...  
141  127.017519  37.258167  MULTILINESTRING ((127.01703 37.25747, 127.0172...  
142  127.017519  37.258167  MULTILINESTRING ((127.01872 37.26071, 127.0186...  
143  127.017519  37.258167  MULTILINESTRING ((127.01703 37.25747, 127.0168...  

[139 rows x 9 columns]

시각화를 위해 법정경계 데이터 가져오기

In [171]:
df29 = gpd.read_file("29.수원시_법정경계(시군구).geojson") 
df29.head()

SIG_CD SIG_KOR_NM                                           geometry
0  41111    수원시 장안구  MULTIPOLYGON (((127.01976 37.35104, 127.02008 ...
1  41113    수원시 권선구  MULTIPOLYGON (((127.03723 37.23684, 127.03679 ...
2  41115    수원시 팔달구  MULTIPOLYGON (((127.03582 37.29361, 127.03644 ...
3  41117    수원시 영통구  MULTIPOLYGON (((127.07484 37.26588, 127.07488 ...

## 시각화 결과

### 수원시 도로 분포

In [172]:
x0, x1 = list(), list()
for lines in linkid['geometry']:
    for line in lines:
        for _x0, _y0, _x1, _y1 in zip(line.xy[0][:-1], line.xy[1][:-1], line.xy[0][1:], line.xy[1][1:]):
            x0.append([_x0, _y0])
            x1.append([_x1, _y1])

lines = shapely.ops.cascaded_union([shapely.geometry.LineString([x0, x1]) for x0, x1 in zip(x0, x1)])

m = folium.Map(
    location=[
        linkid['geometry'].apply(lambda x: x.centroid).y.mean(),
        linkid['geometry'].apply(lambda x: x.centroid).x.mean(),
    ],
    zoom_start=12,
)

folium.Choropleth(
    geo_data=lines,
    line_color='blue',
).add_to(m)

m

### 수원시 버스정류장 분포

In [173]:
center = [37.26404308610561, 127.02347288584993]
m = folium.Map(location=center, zoom_start=13,tiles='cartodbpositron')
style1 = {'fillColor': False, 'color': '#475974'}
style2 = {'fillColor': '#e3fc03', 'color': '#81f0b7'}

folium.GeoJson(data=df29["geometry"], style_function=lambda x:style1).add_to(m) #수원시 구별 경계
for i in range(len(bus_raw)):
    folium.CircleMarker(
        location=[bus_raw.iloc[i]['lat'], bus_raw.iloc[i]['lon']],
        color='green',
        fill=True,
        fill_color = 'green',
        radius=5
).add_to(m)
m

### 전체교통량 상위 30개 버스정류소 위치 분포

In [174]:
center = [37.26404308610561, 127.02347288584993]
m = folium.Map(location=center, zoom_start=13,tiles='cartodbpositron')
style1 = {'fillColor': False, 'color': '#475974'}
style2 = {'fillColor': '#e3fc03', 'color': '#81f0b7'}

folium.GeoJson(data=df29["geometry"], style_function=lambda x:style1).add_to(m) #수원시 구별 경계
for i in range(len(total_top30)):
    folium.CircleMarker(
        location=[total_top30.iloc[i]['lat'], total_top30.iloc[i]['lon']],
        color='#5853A0',
        fill=True,
        fill_color = '',
        radius=8
).add_to(m)
    
x0, x1 = list(), list()
for lines in total_top30_road['geometry']:
    for line in lines:
        for _x0, _y0, _x1, _y1 in zip(line.xy[0][:-1], line.xy[1][:-1], line.xy[0][1:], line.xy[1][1:]):
            x0.append([_x0, _y0])
            x1.append([_x1, _y1])

lines = shapely.ops.cascaded_union([shapely.geometry.LineString([x0, x1]) for x0, x1 in zip(x0, x1)])
folium.Choropleth(
    geo_data=lines,
    line_color='#1E2089',
).add_to(m)
    
m

### 전체교통량 상위 30개 버스정류소 위치 분포

In [175]:
center = [37.26404308610561, 127.02347288584993]
m = folium.Map(location=center, zoom_start=13,tiles='cartodbpositron')
style1 = {'fillColor': False, 'color': '#475974'}
style2 = {'fillColor': '#e3fc03', 'color': '#81f0b7'}

folium.GeoJson(data=df29["geometry"], style_function=lambda x:style1).add_to(m) #수원시 구별 경계
for i in range(len(truck_top30)):
    folium.CircleMarker(
        location=[truck_top30.iloc[i]['lat'], truck_top30.iloc[i]['lon']],
        color='#DF1616',
        fill=True,
        fill_color = '#DF1616',
        radius=8
).add_to(m)
    
x0, x1 = list(), list()
for lines in truck_top30_road['geometry']:
    for line in lines:
        for _x0, _y0, _x1, _y1 in zip(line.xy[0][:-1], line.xy[1][:-1], line.xy[0][1:], line.xy[1][1:]):
            x0.append([_x0, _y0])
            x1.append([_x1, _y1])

lines = shapely.ops.cascaded_union([shapely.geometry.LineString([x0, x1]) for x0, x1 in zip(x0, x1)])
folium.Choropleth(
    geo_data=lines,
    line_color='#BA1313',
).add_to(m)
m

### 전체교통량 하위 30개 버스정류소 위치 분포

In [176]:
center = [37.26404308610561, 127.02347288584993]
m = folium.Map(location=center, zoom_start=13,tiles='cartodbpositron')
style1 = {'fillColor': False, 'color': '#475974'}
style2 = {'fillColor': '#e3fc03', 'color': '#81f0b7'}

folium.GeoJson(data=df29["geometry"], style_function=lambda x:style1).add_to(m) #수원시 구별 경계
for i in range(len(total_bottom30)):
    folium.CircleMarker(
        location=[total_bottom30.iloc[i]['lat'], total_bottom30.iloc[i]['lon']],
        color='#5853A0',
        fill=True,
        fill_color = '',
        radius=8
).add_to(m)
    
x0, x1 = list(), list()
for lines in total_bottom30_road['geometry']:
    for line in lines:
        for _x0, _y0, _x1, _y1 in zip(line.xy[0][:-1], line.xy[1][:-1], line.xy[0][1:], line.xy[1][1:]):
            x0.append([_x0, _y0])
            x1.append([_x1, _y1])

lines = shapely.ops.cascaded_union([shapely.geometry.LineString([x0, x1]) for x0, x1 in zip(x0, x1)])
folium.Choropleth(
    geo_data=lines,
    line_color='#1E2089',
).add_to(m)
    
m

## (3) 최종결합

## 미세먼지노출지수= (미세먼지위험 심도)*(시민들의 노출 빈도)
* 심도:(교통량, 도로재비산먼지, 사업장배출)
* 빈도:(승차인원, 배차간격)

 * factory=정류소 반경 5km이내 대기오염배출사업장 배출밀도  
     배출오염질량(g) / ((정류소와의거리^2)*원주율) 밀도의 개념으로 접근, 단위면적당 오염배출물질
     
 * road= 도로재비산농도/최근접도시측정소농도(동일측정시각)  
   도로재비산농도는 측정일자, 시간이 비정기적이라 계절,날씨,외부적요인 통제하기위해  
   최근접도시측정소농도를 활용함
   
 * traffic_amount= 정류소별 반경 500m 이내 교통량 총합, 일반 자동차대비 화물차 대기오염물질 배출량이 40배라서 화물차 교통량에 40배 해줌.
 
 
 * freq=정류소별 sum(노선별승차인원*노선별배차간격)


#### 전처리(min-max정규화, 결측치 median)

* 심도

In [177]:
traffic_amount=pd.read_csv('data/busTraffic_final.txt',sep=',')
road=pd.read_csv('data/버스정류장별_도로재비산먼지심각도.txt',sep=',',encoding='cp949')
factory=gpd.read_file('data/버스정류장별공장.geojson')

In [178]:
traffic_amount

정류장ID  일반차_추정교통량  화물차_추정교통량_x40
0    200000005   108265.0      1209720.0
1    200000006   115059.0      1196840.0
2    200000007   106545.0      1198280.0
3    200000008        0.0            0.0
4    200000009   417213.0      4857040.0
..         ...        ...            ...
923  203000424   410549.0      3051920.0
924  203000438   332146.0      2614760.0
925  203000439   330558.0      2606640.0
926  203000449    91937.0       828720.0
927  203000450   459149.0      3111000.0

[928 rows x 3 columns]

* 빈도

In [179]:
freq=pd.read_csv('data/dustindex.csv')

- factory는 이미 min-max 정규화 한 상태
- 도로재비산심각도 min-max정규화

In [180]:
road_max=road.도로재비산심각도.max()
road_min=road.도로재비산심각도.min()
road.도로재비산심각도=(road.도로재비산심각도-road_min)/(road_max-road_min)

- 양극단이 심함, 결측 71개행 중앙값으로 대체

In [181]:
road.loc[road.도로재비산심각도.isnull(),'도로재비산심각도']=road.도로재비산심각도.median()

- 정류소별 교통량 총합, min-max정규화

In [182]:
traffic_amount['총합']=traffic_amount['일반차_추정교통량']+traffic_amount['화물차_추정교통량_x40']

traffic_max=traffic_amount['총합'].max()
traffic_min=traffic_amount['총합'].min()

traffic_amount['총합']=(traffic_amount['총합']-traffic_min)/(traffic_max-traffic_min)

### 지수 만들기: 심도 * 빈도

#### 심도: 2020년 수원시 미세먼지 관리 종합계획 수립 연구(공청회 자료) 참고

##### 1. 도로이동오염원: 25.8%
##### 2. 도로재비산먼지:26.1%*62%= 16.2%
##### 3. 사업체연소:4.2%+7.2%+0.2%=11.6%
* 제외: 건설공사: 비도로이동오염원의 90%, 비산먼지 30%를 차지하지만 건설 계획에따라 큰 차이가 나기 때문에 예측 변수로 부적절

- 심도 가중치 설정

In [183]:
weight=np.array([25.8,16.2,11.6])
weight=weight/sum(weight)
weight

array([0.48134328, 0.30223881, 0.21641791])

In [184]:
factory

정류장ID         위치(동) 환승역타입        공장오염밀도  최소최대정규화오염도  \
0    203000237  경기도 수원시 매탄3동    일반  1.077954e+06    1.000000   
1    203000240  경기도 수원시 매탄3동    일반  8.400680e+05    0.779317   
2    203000385  경기도 수원시 매탄3동    일반  8.077172e+05    0.749306   
3    203000130   경기도 수원시 태장동    일반  6.788980e+05    0.629802   
4    203000384   경기도 수원시 태장동    일반  6.729817e+05    0.624314   
..         ...           ...   ...           ...         ...   
923  203000322  경기도 수원시 광교1동    일반  0.000000e+00    0.000000   
924  201000173   경기도 수원시 구운동    일반  0.000000e+00    0.000000   
925  203000263  경기도 수원시 광교1동    일반  0.000000e+00    0.000000   
926  201000172   경기도 수원시 구운동    일반  0.000000e+00    0.000000   
927  200000246  경기도 수원시 조원2동    일반  0.000000e+00    0.000000   

                       geometry  
0    POINT (127.04950 37.24856)  
1    POINT (127.04914 37.24857)  
2    POINT (127.04360 37.25053)  
3    POINT (127.04707 37.24822)  
4    POINT (127.04335 37.25027)  
..                          ...  
923  POINT (127.05144 37.29104)  
924  POINT (126.97056 37.27427)  
925  POINT (127.04734 37.30386)  
926  POINT (126.97464 37.27526)  
927  POINT (127.01474 37.30619)  

[928 rows x 6 columns]

- 데이터결합: 교통량, 도로재비산,사업장

In [185]:
severity=pd.merge(traffic_amount[['정류장ID','총합']],road,how='left',on='정류장ID')
severity=pd.merge(severity,factory[['정류장ID','최소최대정규화오염도','geometry','위치(동)']],how='left',on='정류장ID')
severity.head(5)

정류장ID        총합  도로재비산심각도  최소최대정규화오염도                    geometry  \
0  200000005  0.118314  0.017981         0.0  POINT (127.03155 37.30146)   
1  200000006  0.117768  0.376808         0.0  POINT (127.02946 37.30001)   
2  200000007  0.117133  0.007818         0.0  POINT (127.03129 37.30145)   
3  200000008  0.000000  0.048471         0.0  POINT (127.02770 37.30895)   
4  200000009  0.473464  0.186249         0.0  POINT (127.02093 37.31324)   

         위치(동)  
0  경기도 수원시 연무동  
1  경기도 수원시 연무동  
2  경기도 수원시 연무동  
3  경기도 수원시 연무동  
4  경기도 수원시 연무동

In [186]:
severity.loc[:,'총합':'최소최대정규화오염도'].head(5)

총합  도로재비산심각도  최소최대정규화오염도
0  0.118314  0.017981         0.0
1  0.117768  0.376808         0.0
2  0.117133  0.007818         0.0
3  0.000000  0.048471         0.0
4  0.473464  0.186249         0.0

- 가중치 곱해서 지수 구하기

In [187]:
severity['sev']=np.apply_along_axis(sum,1,severity.loc[:,'총합':'최소최대정규화오염도']*weight)
severity.head(5)

정류장ID        총합  도로재비산심각도  최소최대정규화오염도                    geometry  \
0  200000005  0.118314  0.017981         0.0  POINT (127.03155 37.30146)   
1  200000006  0.117768  0.376808         0.0  POINT (127.02946 37.30001)   
2  200000007  0.117133  0.007818         0.0  POINT (127.03129 37.30145)   
3  200000008  0.000000  0.048471         0.0  POINT (127.02770 37.30895)   
4  200000009  0.473464  0.186249         0.0  POINT (127.02093 37.31324)   

         위치(동)       sev  
0  경기도 수원시 연무동  0.062384  
1  경기도 수원시 연무동  0.170573  
2  경기도 수원시 연무동  0.058744  
3  경기도 수원시 연무동  0.014650  
4  경기도 수원시 연무동  0.284190

- 노출빈도 결합

In [188]:
dustExposure=pd.merge(severity,freq,how='inner',left_on='정류장ID',right_on='정류소ID')

In [189]:
dustExposure['Exposure']=dustExposure.sev*dustExposure.total
dustExposure=dustExposure.sort_values(by='Exposure',ascending=False)

dustExposure.head(5)

정류장ID        총합  도로재비산심각도  최소최대정규화오염도                    geometry  \
612  202000219  0.419152  0.175779    0.001522  POINT (127.00093 37.26746)   
617  202000224  0.288980  0.264078    0.001582  POINT (127.00341 37.26731)   
601  202000208  0.668458  0.050646    0.001330  POINT (126.99961 37.26811)   
766  203000150  0.549781  0.176911    0.079344  POINT (127.05619 37.24473)   
592  202000106  0.424874  0.175779    0.001549  POINT (127.00090 37.26737)   

           위치(동)       sev      정류소ID     total      Exposure  
612  경기도 수원시 매산동  0.255213  202000219  46599661  1.189282e+07  
617  경기도 수원시 매산동  0.219256  202000224  47134286  1.033446e+07  
601  경기도 수원시 매산동  0.337353  202000208  28595407  9.646735e+06  
766  경기도 수원시 태장동  0.335274  203000150  26545428  8.899994e+06  
592  경기도 수원시 매산동  0.257973  202000106  32378188  8.352690e+06

- 저장하기

In [190]:
dustExposure.to_csv('data/미세먼지저감효과.csv',index=False)

In [191]:
dustExposure=pd.read_csv('data/미세먼지저감효과.csv')

In [192]:
soowon_SIGUN=gpd.read_file('29.수원시_법정경계(시군구).geojson')
soowon_DONG=gpd.read_file('30.수원시_법정경계(읍면동).geojson')

In [193]:
geom_dustindex=np.array([re.findall('[0-9]+[.][0-9]+',i) for i in dustExposure.geometry])
geom_dustindex.T[0].astype('float')

array([127.0009335, 127.0034063, 126.9996078, 127.0561861, 127.0009035,
       127.0014835, 127.0423849, 127.0325139, 127.0015854, 127.0324467,
       127.0335522, 127.0447182, 127.0162585, 127.0311642, 127.0472182,
       126.9992795, 127.0309214, 127.0333947, 127.0114299, 127.0554899,
       127.011444 , 127.0151122, 127.0018454, 127.017306 , 127.031487 ,
       127.0437249, 127.051131 , 127.0070258, 127.0465362, 127.0133536,
       127.0147048, 127.044118 , 127.0144093, 127.0152111, 127.0739234,
       126.9989374, 127.0680567, 127.0304022, 126.9668655, 126.984994 ,
       127.0077627, 127.021449 , 126.9920601, 127.013146 , 127.0216746,
       126.9799297, 127.0578015, 127.0271832, 127.0223714, 127.0185408,
       126.9985033, 127.0197156, 127.0013544, 127.0111727, 127.0001122,
       126.9989684, 127.066595 , 127.016805 , 127.0352026, 127.0824245,
       127.0180991, 127.0010387, 127.0353134, 127.0113413, 127.0293924,
       127.0339336, 127.0158561, 127.0128416, 127.05661  , 126.9

In [194]:
dustExposure.columns

Index(['정류장ID', '총합', '도로재비산심각도', '최소최대정규화오염도', 'geometry', '위치(동)', 'sev',
       '정류소ID', 'total', 'Exposure'],
      dtype='object')

- df => geodataframe 변환

In [195]:
geom_dustindex=np.array([re.findall('[0-9]+[.][0-9]+',i) for i in dustExposure.geometry])

geometry_dust=gpd.points_from_xy(geom_dustindex.T[0].astype('float'),geom_dustindex.T[1].astype('float'))
cols=['정류장ID', '총합', '도로재비산심각도', '최소최대정규화오염도','위치(동)', 'sev', 'total', 'Exposure']
dustExposure=gpd.GeoDataFrame(dustExposure[cols], geometry=geometry_dust,
                         crs=from_epsg(4326))
dustExposure.index=range(len(dustExposure))
dustExposure.tail(5)

/opt/app-root/lib/python3.6/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


정류장ID        총합  도로재비산심각도  최소최대정규화오염도         위치(동)       sev  total  \
914  201000391  0.155830  0.222097    0.012023    경기도 수원시 평동  0.144736   1310   
915  203000308  0.248861  0.133506    0.015949  경기도 수원시 광교1동  0.163590   1040   
916  203000438  0.264541  0.062335    0.000000  경기도 수원시 광교1동  0.146175   1126   
917  201000393  0.258489  0.222097    0.013152    경기도 수원시 평동  0.194394    625   
918  201000326  0.422545  0.222097    0.009783    경기도 수원시 평동  0.272633     50   

       Exposure                    geometry  
914  189.604235  POINT (126.96913 37.24291)  
915  170.133893  POINT (127.06672 37.27545)  
916  164.592796  POINT (127.05620 37.29492)  
917  121.496550  POINT (126.96978 37.24117)  
918   13.631634  POINT (126.96817 37.24551)

In [196]:
dustExposure_position=[[i.y,i.x] for i in dustExposure.geometry]

In [197]:
dustExposure.to_file('data/통합지수.geojson',driver='GeoJSON',index=False)
dustExposure

정류장ID        총합  도로재비산심각도  최소최대정규화오염도         위치(동)       sev  \
0    202000219  0.419152  0.175779    0.001522   경기도 수원시 매산동  0.255213   
1    202000224  0.288980  0.264078    0.001582   경기도 수원시 매산동  0.219256   
2    202000208  0.668458  0.050646    0.001330   경기도 수원시 매산동  0.337353   
3    203000150  0.549781  0.176911    0.079344   경기도 수원시 태장동  0.335274   
4    202000106  0.424874  0.175779    0.001549   경기도 수원시 매산동  0.257973   
..         ...       ...       ...         ...           ...       ...   
914  201000391  0.155830  0.222097    0.012023    경기도 수원시 평동  0.144736   
915  203000308  0.248861  0.133506    0.015949  경기도 수원시 광교1동  0.163590   
916  203000438  0.264541  0.062335    0.000000  경기도 수원시 광교1동  0.146175   
917  201000393  0.258489  0.222097    0.013152    경기도 수원시 평동  0.194394   
918  201000326  0.422545  0.222097    0.009783    경기도 수원시 평동  0.272633   

        total      Exposure                    geometry  
0    46599661  1.189282e+07  POINT (127.00093 37.26746)  
1    47134286  1.033446e+07  POINT (127.00341 37.26731)  
2    28595407  9.646735e+06  POINT (126.99961 37.26811)  
3    26545428  8.899994e+06  POINT (127.05619 37.24473)  
4    32378188  8.352690e+06  POINT (127.00090 37.26737)  
..        ...           ...                         ...  
914      1310  1.896042e+02  POINT (126.96913 37.24291)  
915      1040  1.701339e+02  POINT (127.06672 37.27545)  
916      1126  1.645928e+02  POINT (127.05620 37.29492)  
917       625  1.214965e+02  POINT (126.96978 37.24117)  
918        50  1.363163e+01  POINT (126.96817 37.24551)  

[919 rows x 9 columns]

## (4) 시각화

### 미세먼지 노출지수 top100 시각화

In [198]:
# 미세먼지노출지수 top100 시각화
m=folium.Map([37.281796, 127.016808],zoom_start=12)

data=dustExposure_position[:50]
popups=[str(i)+': '+str(dustExposure['Exposure'][i]) for i in range(len(dustExposure[:50]))]

plugins.MarkerCluster(data,popups=popups).add_to(m)
    
folium.GeoJson(
    soowon_SIGUN,
    name='soowon_municipalities',
).add_to(m)
m

### 미세먼지 노출지수 top30 시각화

In [199]:
m=folium.Map([37.281796, 127.016808],zoom_start=12)

for i in range(len(dustExposure_position[:30])):
    df=pd.DataFrame(dustExposure.loc[i,['정류장ID','Exposure']])
    html=df.to_html(classes='table table-striped table-hover table-condensed table-responsive')
    
    icon_number=plugins.BeautifyIcon(
    number=i,
    border_color='darkred',
    text_color='darkred',
    inner_icon_style='margin-top:0;')
    
    folium.Marker(
    location=dustExposure_position[i],
    popup=html,
    icon=icon_number).add_to(m)
    
    
folium.GeoJson(
    soowon_SIGUN,
    name='soowon_municipalities',
    
).add_to(m)
m